# インポート

In [49]:
import pandas as pd
import pickle

import numpy as np

import scraping
import create_rawdf
import create_prediction_population
from pathlib import Path
import requests

import scraping_prediction

DATA_DIR = Path("..", "data")
HTML_RACE_DIR = DATA_DIR / "html" / "race"
HTML_HORSE_DIR = DATA_DIR / "html" / "horse"
HTML_PED_DIR = DATA_DIR / "html" / "ped"
HTML_LEADING_DIR = DATA_DIR / "html" / "leading"
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
# モジュールの変更を反映させたい時実行
%autoreload

# データ取得

In [51]:
#全部載せ
# 一時保存ディレクトリ
TMP_DIR = scraping.DATA_DIR / "tmp"

In [52]:
# 開催日一覧の取得
kaisai_date_list = scraping.scrape_kaisai_date(
    from_="2024-12", to_="2024-12", save_dir=TMP_DIR
)

100%|████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.28s/it]


In [ ]:
# 途中まで処理したlistからスクレイピングするための前処理
def load_kaisai_date_list_2(file_path: Path) -> list[str]:
    with open(file_path, "r") as f:
        return f.read().splitlines()

kaisai_date_file = Path(TMP_DIR / "kaisai_date_list_2.txt")
kaisai_date_list_2 = load_kaisai_date_list_2(kaisai_date_file)
#全て終わった場合、kaisai_date_listにkaisai_date_list_2の内容をくっつけること

In [53]:
# スクレイピング対象レースのid取得
race_id_list = scraping.scrape_race_id_list(
    kaisai_date_list, save_dir=TMP_DIR
)

100%|████████████████████████████████████████████████████████████████████| 8/8 [00:22<00:00,  2.82s/it]


In [ ]:
# notebookの再起動などでrace_id_listが失われた場合はファイルから読み込む
with open(TMP_DIR / "race_id_list.txt", "r") as f:
    race_id_list = f.read().splitlines()

In [54]:
len(race_id_list)

252

In [113]:
race_id_list

['201606010101',
 '201606010102',
 '201606010103',
 '201606010104',
 '201606010105',
 '201606010106',
 '201606010107',
 '201606010108',
 '201606010109',
 '201606010110',
 '201606010111',
 '201606010112',
 '201608010101',
 '201608010102',
 '201608010103',
 '201608010104',
 '201608010105',
 '201608010106',
 '201608010107',
 '201608010108',
 '201608010109',
 '201608010110',
 '201608010111',
 '201608010112',
 '201606010201',
 '201606010202',
 '201606010203',
 '201606010204',
 '201606010205',
 '201606010206',
 '201606010207',
 '201606010208',
 '201606010209',
 '201606010210',
 '201606010211',
 '201606010212',
 '201608010201',
 '201608010202',
 '201608010203',
 '201608010204',
 '201608010205',
 '201608010206',
 '201608010207',
 '201608010208',
 '201608010209',
 '201608010210',
 '201608010211',
 '201608010212',
 '201606010301',
 '201606010302',
 '201606010303',
 '201606010304',
 '201606010305',
 '201606010306',
 '201606010307',
 '201606010308',
 '201606010309',
 '201606010310',
 '201606010311

In [19]:
#すでにあるレースを手動で取ってくる方法
race_id_list2 = """202406010101
202406010102
202406010103
202406010104
202406010105
202406010106
202406010107
202406010108
202406010109
202406010110
202406010111
202406010112
202408010101
202408010102
202408010103
202408010104
202408010105
202408010106
202408010107
202408010108
202408010109
202408010110
202408010111
202408010112
202406010201
202406010202
202406010203
202406010204
202406010205
202406010206
202406010207
202406010208
202406010209
202406010210
202406010211
202406010212
202408010201
202408010202
202408010203
202408010204
202408010205
202408010206
202408010207
202408010208
202408010209
202408010210
202408010211
202408010212
202406010301
202406010302
202406010303
202406010304
202406010305
202406010306
202406010307
202406010308
202406010309
202406010310
202406010311
202406010312
202408010301
202408010302
202408010303
202408010304
202408010305
202408010306
202408010307
202408010308
202408010309
202408010310
202408010311
202408010312
202406010401
202406010402
202406010403
202406010404
202406010405
202406010406
202406010407
202406010408
202406010409
202406010410
202406010411
202406010412
202408010401
202408010402
202408010403
202408010404
202408010405
202408010406
202408010407
202408010408
202408010409
202408010410
202408010411
202408010412
202410010101
202410010102
202410010103
202410010104
202410010105
202410010106
202410010107
202410010108
202410010109
202410010110
202410010111
202410010112
202406010501
202406010502
202406010503
202406010504
202406010505
202406010506
202406010507
202406010508
202406010509
202406010510
202406010511
202406010512
202408010501
202408010502
202408010503
202408010504
202408010505
202408010506
202408010507
202408010508
202408010509
202408010510
202408010511
202408010512
202410010201
202410010202
202410010203
202410010204
202410010205
202410010206
202410010207
202410010208
202410010209
202410010210
202410010211
202410010212
202406010601
202406010602
202406010603
202406010604
202406010605
202406010606
202406010607
202406010608
202406010609
202406010610
202406010611
202406010612
202408010601
202408010602
202408010603
202408010604
202408010605
202408010606
202408010607
202408010608
202408010609
202408010610
202408010611
202408010612
202410010301
202410010302
202410010303
202410010304
202410010305
202410010306
202410010307
202410010308
202410010309
202410010310
202410010311
202410010312
202406010701
202406010702
202406010703
202406010704
202406010705
202406010706
202406010707
202406010708
202406010709
202406010710
202406010711
202406010712
202408010701
202408010702
202408010703
202408010704
202408010705
202408010706
202408010707
202408010708
202408010709
202408010710
202408010711
202408010712
202410010401
202410010402
202410010403
202410010404
202410010405
202410010406
202410010407
202410010408
202410010409
202410010410
202410010411
202410010412
202405010101
202405010102
202405010103
202405010104
202405010105
202405010106
202405010107
202405010108
202405010109
202405010110
202405010111
202405010112
202408020101
202408020102
202408020103
202408020104
202408020105
202408020106
202408020107
202408020108
202408020109
202408020110
202408020111
202408020112
202410010501
202410010502
202410010503
202410010504
202410010505
202410010506
202410010507
202410010508
202410010509
202410010510
202410010511
202410010512
202405010201
202405010202
202405010203
202405010204
202405010205
202405010206
202405010207
202405010208
202405010209
202405010210
202405010211
202405010212
202408020201
202408020202
202408020203
202408020204
202408020205
202408020206
202408020207
202408020208
202408020209
202408020210
202408020211
202408020212
202410010601
202410010602
202410010603
202410010604
202410010605
202410010606
202410010607
202410010608
202410010609
202410010610
202410010611
202410010612
202405010301
202405010302
202405010303
202405010304
202405010305
202405010306
202405010307
202405010308
202405010309
202405010310
202405010311
202405010312
202408020301
202408020302
202408020303
202408020304
202408020305
202408020306
202408020307
202408020308
202408020309
202408020310
202408020311
202408020312
202410010701
202410010702
202410010703
202410010704
202410010705
202410010706
202410010707
202410010708
202410010709
202410010710
202410010711
202410010712
202405010401
202405010402
202405010403
202405010404
202405010405
202405010406
202405010407
202405010408
202405010409
202405010410
202405010411
202405010412
202408020401
202408020402
202408020403
202408020404
202408020405
202408020406
202408020407
202408020408
202408020409
202408020410
202408020411
202408020412
202410010801
202410010802
202410010803
202410010804
202410010805
202410010806
202410010807
202410010808
202410010809
202410010810
202410010811
202410010812
202405010501
202405010502
202405010503
202405010504
202405010505
202405010506
202405010507
202405010508
202405010509
202405010510
202405010511
202405010512
202408020501
202408020502
202408020503
202408020504
202408020505
202408020506
202408020507
202408020508
202408020509
202408020510
202408020511
202408020512
202410020101
202410020102
202410020103
202410020104
202410020105
202410020106
202410020107
202410020108
202410020109
202410020110
202410020111
202410020112
202405010601
202405010602
202405010603
202405010604
202405010605
202405010606
202405010607
202405010608
202405010609
202405010610
202405010611
202405010612
202408020601
202408020602
202408020603
202408020604
202408020605
202408020606
202408020607
202408020608
202408020609
202408020610
202408020611
202408020612
202410020201
202410020202
202410020203
202410020204
202410020205
202410020206
202410020207
202410020208
202410020209
202410020210
202410020211
202410020212
202405010701
202405010702
202405010703
202405010704
202405010705
202405010706
202405010707
202405010708
202405010709
202405010710
202405010711
202405010712
202408020701
202408020702
202408020703
202408020704
202408020705
202408020706
202408020707
202408020708
202408020709
202408020710
202408020711
202408020712
202410020301
202410020302
202410020303
202410020304
202410020305
202410020306
202410020307
202410020308
202410020309
202410020310
202410020311
202410020312
202405010801
202405010802
202405010803
202405010804
202405010805
202405010806
202405010807
202405010808
202405010809
202405010810
202405010811
202405010812
202408020801
202408020802
202408020803
202408020804
202408020805
202408020806
202408020807
202408020808
202408020809
202408020810
202408020811
202408020812
202410020401
202410020402
202410020403
202410020404
202410020405
202410020406
202410020407
202410020408
202410020409
202410020410
202410020411
202410020412
202406020101
202406020102
202406020103
202406020104
202406020105
202406020106
202406020107
202406020108
202406020109
202406020110
202406020111
202406020112
202409010101
202409010102
202409010103
202409010104
202409010105
202409010106
202409010107
202409010108
202409010109
202409010110
202409010111
202409010112
202410020501
202410020502
202410020503
202410020504
202410020505
202410020506
202410020507
202410020508
202410020509
202410020510
202410020511
202410020512
202406020201
202406020202
202406020203
202406020204
202406020205
202406020206
202406020207
202406020208
202406020209
202406020210
202406020211
202406020212
202409010201
202409010202
202409010203
202409010204
202409010205
202409010206
202409010207
202409010208
202409010209
202409010210
202409010211
202409010212
202410020601
202410020602
202410020603
202410020604
202410020605
202410020606
202410020607
202410020608
202410020609
202410020610
202410020611
202410020612
202406020301
202406020302
202406020303
202406020304
202406020305
202406020306
202406020307
202406020308
202406020309
202406020310
202406020311
202406020312
202409010301
202409010302
202409010303
202409010304
202409010305
202409010306
202409010307
202409010308
202409010309
202409010310
202409010311
202409010312
202410020701
202410020702
202410020703
202410020704
202410020705
202410020706
202410020707
202410020708
202410020709
202410020710
202410020711
202410020712
202406020401
202406020402
202406020403
202406020404
202406020405
202406020406
202406020407
202406020408
202406020409
202406020410
202406020411
202406020412
202409010401
202409010402
202409010403
202409010404
202409010405
202409010406
202409010407
202409010408
202409010409
202409010410
202409010411
202409010412
202410020801
202410020802
202410020803
202410020804
202410020805
202410020806
202410020807
202410020808
202410020809
202410020810
202410020811
202410020812
202406020501
202406020502
202406020503
202406020504
202406020505
202406020506
202406020507
202406020508
202406020509
202406020510
202406020511
202406020512
202409010501
202409010502
202409010503
202409010504
202409010505
202409010506
202409010507
202409010508
202409010509
202409010510
202409010511
202409010512
202407010101
202407010102
202407010103
202407010104
202407010105
202407010106
202407010107
202407010108
202407010109
202407010110
202407010111
202407010112
202406020601
202406020602
202406020603
202406020604
202406020605
202406020606
202406020607
202406020608
202406020609
202406020610
202406020611
202406020612
202409010601
202409010602
202409010603
202409010604
202409010605
202409010606
202409010607
202409010608
202409010609
202409010610
202409010611
202409010612
202407010201
202407010202
202407010203
202407010204
202407010205
202407010206
202407010207
202407010208
202407010209
202407010210
202407010211
202407010212
202406020701
202406020702
202406020703
202406020704
202406020705
202406020706
202406020707
202406020708
202406020709
202406020710
202406020711
202406020712
202409010701
202409010702
202409010703
202409010704
202409010705
202409010706
202409010707
202409010708
202409010709
202409010710
202409010711
202409010712
202407010301
202407010302
202407010303
202407010304
202407010305
202407010306
202407010307
202407010308
202407010309
202407010310
202407010311
202407010312
202406020801
202406020802
202406020803
202406020804
202406020805
202406020806
202406020807
202406020808
202406020809
202406020810
202406020811
202406020812
202409010801
202409010802
202409010803
202409010804
202409010805
202409010806
202409010807
202409010808
202409010809
202409010810
202409010811
202409010812
202407010401
202407010402
202407010403
202407010404
202407010405
202407010406
202407010407
202407010408
202407010409
202407010410
202407010411
202407010412
202406030101
202406030102
202406030103
202406030104
202406030105
202406030106
202406030107
202406030108
202406030109
202406030110
202406030111
202406030112
202409020101
202409020102
202409020103
202409020104
202409020105
202409020106
202409020107
202409020108
202409020109
202409020110
202409020111
202409020112
202407010501
202407010502
202407010503
202407010504
202407010505
202407010506
202407010507
202407010508
202407010509
202407010510
202407010511
202407010512
202406030201
202406030202
202406030203
202406030204
202406030205
202406030206
202406030207
202406030208
202406030209
202406030210
202406030211
202406030212
202409020201
202409020202
202409020203
202409020204
202409020205
202409020206
202409020207
202409020208
202409020209
202409020210
202409020211
202409020212
202407010601
202407010602
202407010603
202407010604
202407010605
202407010606
202407010607
202407010608
202407010609
202407010610
202407010611
202407010612
202406030301
202406030302
202406030303
202406030304
202406030305
202406030306
202406030307
202406030308
202406030309
202406030310
202406030311
202406030312
202409020301
202409020302
202409020303
202409020304
202409020305
202409020306
202409020307
202409020308
202409020309
202409020310
202409020311
202409020312
202406030401
202406030402
202406030403
202406030404
202406030405
202406030406
202406030407
202406030408
202406030409
202406030410
202406030411
202406030412
202409020401
202409020402
202409020403
202409020404
202409020405
202409020406
202409020407
202409020408
202409020409
202409020410
202409020411
202409020412
202406030501
202406030502
202406030503
202406030504
202406030505
202406030506
202406030507
202406030508
202406030509
202406030510
202406030511
202406030512
202409020501
202409020502
202409020503
202409020504
202409020505
202409020506
202409020507
202409020508
202409020509
202409020510
202409020511
202409020512
202403010101
202403010102
202403010103
202403010104
202403010105
202403010106
202403010107
202403010108
202403010109
202403010110
202403010111
202403010112
202406030601
202406030602
202406030603
202406030604
202406030605
202406030606
202406030607
202406030608
202406030609
202406030610
202406030611
202406030612
202409020601
202409020602
202409020603
202409020604
202409020605
202409020606
202409020607
202409020608
202409020609
202409020610
202409020611
202409020612
202403010201
202403010202
202403010203
202403010204
202403010205
202403010206
202403010207
202403010208
202403010209
202403010210
202403010211
202403010212
202406030701
202406030702
202406030703
202406030704
202406030705
202406030706
202406030707
202406030708
202406030709
202406030710
202406030711
202406030712
202409020701
202409020702
202409020703
202409020704
202409020705
202409020706
202409020707
202409020708
202409020709
202409020710
202409020711
202409020712
202403010301
202403010302
202403010303
202403010304
202403010305
202403010306
202403010307
202403010308
202403010309
202403010310
202403010311
202403010312
202406030801
202406030802
202406030803
202406030804
202406030805
202406030806
202406030807
202406030808
202406030809
202406030810
202406030811
202406030812
202409020801
202409020802
202409020803
202409020804
202409020805
202409020806
202409020807
202409020808
202409020809
202409020810
202409020811
202409020812
202403010401
202403010402
202403010403
202403010404
202403010405
202403010406
202403010407
202403010408
202403010409
202403010410
202403010411
202403010412
202405020101
202405020102
202405020103
202405020104
202405020105
202405020106
202405020107
202405020108
202405020109
202405020110
202405020111
202405020112
202408030101
202408030102
202408030103
202408030104
202408030105
202408030106
202408030107
202408030108
202408030109
202408030110
202408030111
202408030112
202403010501
202403010502
202403010503
202403010504
202403010505
202403010506
202403010507
202403010508
202403010509
202403010510
202403010511
202403010512
202405020201
202405020202
202405020203
202405020204
202405020205
202405020206
202405020207
202405020208
202405020209
202405020210
202405020211
202405020212
202408030201
202408030202
202408030203
202408030204
202408030205
202408030206
202408030207
202408030208
202408030209
202408030210
202408030211
202408030212
202403010601
202403010602
202403010603
202403010604
202403010605
202403010606
202403010607
202403010608
202403010609
202403010610
202403010611
202403010612
202405020301
202405020302
202405020303
202405020304
202405020305
202405020306
202405020307
202405020308
202405020309
202405020310
202405020311
202405020312
202408030301
202408030302
202408030303
202408030304
202408030305
202408030306
202408030307
202408030308
202408030309
202408030310
202408030311
202408030312
202404010101
202404010102
202404010103
202404010104
202404010105
202404010106
202404010107
202404010108
202404010109
202404010110
202404010111
202404010112
202405020401
202405020402
202405020403
202405020404
202405020405
202405020406
202405020407
202405020408
202405020409
202405020410
202405020411
202405020412
202408030401
202408030402
202408030403
202408030404
202408030405
202408030406
202408030407
202408030408
202408030409
202408030410
202408030411
202408030412
202404010201
202404010202
202404010203
202404010204
202404010205
202404010206
202404010207
202404010208
202404010209
202404010210
202404010211
202404010212
202405020501
202405020502
202405020503
202405020504
202405020505
202405020506
202405020507
202405020508
202405020509
202405020510
202405020511
202405020512
202408030501
202408030502
202408030503
202408030504
202408030505
202408030506
202408030507
202408030508
202408030509
202408030510
202408030511
202408030512
202404010301
202404010302
202404010303
202404010304
202404010305
202404010306
202404010307
202404010308
202404010309
202404010310
202404010311
202404010312
202405020601
202405020602
202405020603
202405020604
202405020605
202405020606
202405020607
202405020608
202405020609
202405020610
202405020611
202405020612
202408030601
202408030602
202408030603
202408030604
202408030605
202408030606
202408030607
202408030608
202408030609
202408030610
202408030611
202408030612
202404010401
202404010402
202404010403
202404010404
202404010405
202404010406
202404010407
202404010408
202404010409
202404010410
202404010411
202404010412
202405020701
202405020702
202405020703
202405020704
202405020705
202405020706
202405020707
202405020708
202405020709
202405020710
202405020711
202405020712
202408030701
202408030702
202408030703
202408030704
202408030705
202408030706
202408030707
202408030708
202408030709
202408030710
202408030711
202408030712
202404010501
202404010502
202404010503
202404010504
202404010505
202404010506
202404010507
202404010508
202404010509
202404010510
202404010511
202404010512
202405020801
202405020802
202405020803
202405020804
202405020805
202405020806
202405020807
202405020808
202405020809
202405020810
202405020811
202405020812
202408030801
202408030802
202408030803
202408030804
202408030805
202408030806
202408030807
202408030808
202408030809
202408030810
202408030811
202408030812
202404010601
202404010602
202404010603
202404010604
202404010605
202404010606
202404010607
202404010608
202404010609
202404010610
202404010611
202404010612
202405020901
202405020902
202405020903
202405020904
202405020905
202405020906
202405020907
202405020908
202405020909
202405020910
202405020911
202405020912
202408030901
202408030902
202408030903
202408030904
202408030905
202408030906
202408030907
202408030908
202408030909
202408030910
202408030911
202408030912
202404010701
202404010702
202404010703
202404010704
202404010705
202404010706
202404010707
202404010708
202404010709
202404010710
202404010711
202404010712
202405021001
202405021002
202405021003
202405021004
202405021005
202405021006
202405021007
202405021008
202405021009
202405021010
202405021011
202405021012
202408031001
202408031002
202408031003
202408031004
202408031005
202408031006
202408031007
202408031008
202408031009
202408031010
202408031011
202408031012
202404010801
202404010802
202404010803
202404010804
202404010805
202404010806
202404010807
202404010808
202404010809
202404010810
202404010811
202404010812
202405021101
202405021102
202405021103
202405021104
202405021105
202405021106
202405021107
202405021108
202405021109
202405021110
202405021111
202405021112
202408031101
202408031102
202408031103
202408031104
202408031105
202408031106
202408031107
202408031108
202408031109
202408031110
202408031111
202408031112
202405021201
202405021202
202405021203
202405021204
202405021205
202405021206
202405021207
202405021208
202405021209
202405021210
202405021211
202405021212
202408031201
202408031202
202408031203
202408031204
202408031205
202408031206
202408031207
202408031208
202408031209
202408031210
202408031211
202408031212
202405030101
202405030102
202405030103
202405030104
202405030105
202405030106
202405030107
202405030108
202405030109
202405030110
202405030111
202405030112
202408040101
202408040102
202408040103
202408040104
202408040105
202408040106
202408040107
202408040108
202408040109
202408040110
202408040111
202408040112
202405030201
202405030202
202405030203
202405030204
202405030205
202405030206
202405030207
202405030208
202405030209
202405030210
202405030211
202405030212
202408040201
202408040202
202408040203
202408040204
202408040205
202408040206
202408040207
202408040208
202408040209
202408040210
202408040211
202408040212
202405030301
202405030302
202405030303
202405030304
202405030305
202405030306
202405030307
202405030308
202405030309
202405030310
202405030311
202405030312
202408040301
202408040302
202408040303
202408040304
202408040305
202408040306
202408040307
202408040308
202408040309
202408040310
202408040311
202408040312
202402010101
202402010102
202402010103
202402010104
202402010105
202402010106
202402010107
202402010108
202402010109
202402010110
202402010111
202402010112
202405030401
202405030402
202405030403
202405030404
202405030405
202405030406
202405030407
202405030408
202405030409
202405030410
202405030411
202405030412
202408040401
202408040402
202408040403
202408040404
202408040405
202408040406
202408040407
202408040408
202408040409
202408040410
202408040411
202408040412
202402010201
202402010202
202402010203
202402010204
202402010205
202402010206
202402010207
202402010208
202402010209
202402010210
202402010211
202402010212
202405030501
202405030502
202405030503
202405030504
202405030505
202405030506
202405030507
202405030508
202405030509
202405030510
202405030511
202405030512
202408040501
202408040502
202408040503
202408040504
202408040505
202408040506
202408040507
202408040508
202408040509
202408040510
202408040511
202408040512
202402010301
202402010302
202402010303
202402010304
202402010305
202402010306
202402010307
202402010308
202402010309
202402010310
202402010311
202402010312
202405030601
202405030602
202405030603
202405030604
202405030605
202405030606
202405030607
202405030608
202405030609
202405030610
202405030611
202405030612
202408040601
202408040602
202408040603
202408040604
202408040605
202408040606
202408040607
202408040608
202408040609
202408040610
202408040611
202408040612
202402010401
202402010402
202402010403
202402010404
202402010405
202402010406
202402010407
202402010408
202402010409
202402010410
202402010411
202402010412
202405030701
202405030702
202405030703
202405030704
202405030705
202405030706
202405030707
202405030708
202405030709
202405030710
202405030711
202405030712
202408040701
202408040702
202408040703
202408040704
202408040705
202408040706
202408040707
202408040708
202408040709
202408040710
202408040711
202408040712
202402010501
202402010502
202402010503
202402010504
202402010505
202402010506
202402010507
202402010508
202402010509
202402010510
202402010511
202402010512
202405030801
202405030802
202405030803
202405030804
202405030805
202405030806
202405030807
202405030808
202405030809
202405030810
202405030811
202405030812
202408040801
202408040802
202408040803
202408040804
202408040805
202408040806
202408040807
202408040808
202408040809
202408040810
202408040811
202408040812
202402010601
202402010602
202402010603
202402010604
202402010605
202402010606
202402010607
202402010608
202402010609
202402010610
202402010611
202402010612
202403020101
202403020102
202403020103
202403020104
202403020105
202403020106
202403020107
202403020108
202403020109
202403020110
202403020111
202403020112
202410030101
202410030102
202410030103
202410030104
202410030105
202410030106
202410030107
202410030108
202410030109
202410030110
202410030111
202410030112
202402010701
202402010702
202402010703
202402010704
202402010705
202402010706
202402010707
202402010708
202402010709
202402010710
202402010711
202402010712
202403020201
202403020202
202403020203
202403020204
202403020205
202403020206
202403020207
202403020208
202403020209
202403020210
202403020211
202403020212
202410030201
202410030202
202410030203
202410030204
202410030205
202410030206
202410030207
202410030208
202410030209
202410030210
202410030211
202410030212
202402010801
202402010802
202402010803
202402010804
202402010805
202402010806
202402010807
202402010808
202402010809
202402010810
202402010811
202402010812
202403020301
202403020302
202403020303
202403020304
202403020305
202403020306
202403020307
202403020308
202403020309
202403020310
202403020311
202403020312
202410030301
202410030302
202410030303
202410030304
202410030305
202410030306
202410030307
202410030308
202410030309
202410030310
202410030311
202410030312
202402010901
202402010902
202402010903
202402010904
202402010905
202402010906
202402010907
202402010908
202402010909
202402010910
202402010911
202402010912
202403020401
202403020402
202403020403
202403020404
202403020405
202403020406
202403020407
202403020408
202403020409
202403020410
202403020411
202403020412
202410030401
202410030402
202410030403
202410030404
202410030405
202410030406
202410030407
202410030408
202410030409
202410030410
202410030411
202410030412
202402011001
202402011002
202402011003
202402011004
202402011005
202402011006
202402011007
202402011008
202402011009
202402011010
202402011011
202402011012
202403020501
202403020502
202403020503
202403020504
202403020505
202403020506
202403020507
202403020508
202403020509
202403020510
202403020511
202403020512
202410030501
202410030502
202410030503
202410030504
202410030505
202410030506
202410030507
202410030508
202410030509
202410030510
202410030511
202410030512
202402011101
202402011102
202402011103
202402011104
202402011105
202402011106
202402011107
202402011108
202402011109
202402011110
202402011111
202402011112
202403020601
202403020602
202403020603
202403020604
202403020605
202403020606
202403020607
202403020608
202403020609
202403020610
202403020611
202403020612
202410030601
202410030602
202410030603
202410030604
202410030605
202410030606
202410030607
202410030608
202410030609
202410030610
202410030611
202410030612
202402011201
202402011202
202402011203
202402011204
202402011205
202402011206
202402011207
202402011208
202402011209
202402011210
202402011211
202402011212
202403020701
202403020702
202403020703
202403020704
202403020705
202403020706
202403020707
202403020708
202403020709
202403020710
202403020711
202403020712
202410030701
202410030702
202410030703
202410030704
202410030705
202410030706
202410030707
202410030708
202410030709
202410030710
202410030711
202410030712
202401010101
202401010102
202401010103
202401010104
202401010105
202401010106
202401010107
202401010108
202401010109
202401010110
202401010111
202401010112
202403020801
202403020802
202403020803
202403020804
202403020805
202403020806
202403020807
202403020808
202403020809
202403020810
202403020811
202403020812
202410030801
202410030802
202410030803
202410030804
202410030805
202410030806
202410030807
202410030808
202410030809
202410030810
202410030811
202410030812
202401010201
202401010202
202401010203
202401010204
202401010205
202401010206
202401010207
202401010208
202401010209
202401010210
202401010211
202401010212
202404020101
202404020102
202404020103
202404020104
202404020105
202404020106
202404020107
202404020108
202404020109
202404020110
202404020111
202404020112
202401010301
202401010302
202401010303
202401010304
202401010305
202401010306
202401010307
202401010308
202401010309
202401010310
202401010311
202401010312
202404020201
202404020202
202404020203
202404020204
202404020205
202404020206
202404020207
202404020208
202404020209
202404020210
202404020211
202404020212
202401010401
202401010402
202401010403
202401010404
202401010405
202401010406
202401010407
202401010408
202401010409
202401010410
202401010411
202401010412
202404020301
202404020302
202404020303
202404020304
202404020305
202404020306
202404020307
202404020308
202404020309
202404020310
202404020311
202404020312
202401010501
202401010502
202401010503
202401010504
202401010505
202401010506
202401010507
202401010508
202401010509
202401010510
202401010511
202401010512
202404020401
202404020402
202404020403
202404020404
202404020405
202404020406
202404020407
202404020408
202404020409
202404020410
202404020411
202404020412
202401010601
202401010602
202401010603
202401010604
202401010605
202401010606
202401010607
202401010608
202401010609
202401010610
202401010611
202401010612
202404030101
202404030102
202404030103
202404030104
202404030105
202404030106
202404030107
202404030108
202404030109
202404030110
202404030111
202404030112
202407020101
202407020102
202407020103
202407020104
202407020105
202407020106
202407020107
202407020108
202407020109
202407020110
202407020111
202407020112
202401020101
202401020102
202401020103
202401020104
202401020105
202401020106
202401020107
202401020108
202401020109
202401020110
202401020111
202401020112
202404030201
202404030202
202404030203
202404030204
202404030205
202404030206
202404030207
202404030208
202404030209
202404030210
202404030211
202404030212
202407020201
202407020202
202407020203
202407020204
202407020205
202407020206
202407020207
202407020208
202407020209
202407020210
202407020211
202407020212
202401020201
202401020202
202401020203
202401020204
202401020205
202401020206
202401020207
202401020208
202401020209
202401020210
202401020211
202401020212
202404030301
202404030302
202404030303
202404030304
202404030305
202404030306
202404030307
202404030308
202404030309
202404030310
202404030311
202404030312
202407020301
202407020302
202407020303
202407020304
202407020305
202407020306
202407020307
202407020308
202407020309
202407020310
202407020311
202407020312
202401020301
202401020302
202401020303
202401020304
202401020305
202401020306
202401020307
202401020308
202401020309
202401020310
202401020311
202401020312
202404030401
202404030402
202404030403
202404030404
202404030405
202404030406
202404030407
202404030408
202404030409
202404030410
202404030411
202404030412
202407020401
202407020402
202407020403
202407020404
202407020405
202407020406
202407020407
202407020408
202407020409
202407020410
202407020411
202407020412
202401020401
202401020402
202401020403
202401020404
202401020405
202401020406
202401020407
202401020408
202401020409
202401020410
202401020411
202401020412
202404030501
202404030502
202404030503
202404030504
202404030505
202404030506
202404030507
202404030508
202404030509
202404030510
202404030511
202404030512
202407020501
202407020502
202407020503
202407020504
202407020505
202407020506
202407020507
202407020508
202407020509
202407020510
202407020511
202407020512
202401020501
202401020502
202401020503
202401020504
202401020505
202401020506
202401020507
202401020508
202401020509
202401020510
202401020511
202401020512
202404030601
202404030602
202404030603
202404030604
202404030605
202404030606
202404030607
202404030608
202404030609
202404030610
202404030611
202407020601
202407020602
202407020603
202407020604
202407020605
202407020606
202407020607
202407020608
202407020609
202407020610
202407020611
202407020612
202401020601
202401020602
202401020603
202401020604
202401020605
202401020606
202401020607
202401020608
202401020609
202401020610
202401020611
202401020612
202404030701
202404030702
202404030703
202404030704
202404030705
202404030706
202404030707
202404030708
202404030709
202404030710
202404030711
202404030712
202407020701
202407020702
202407020703
202407020704
202407020705
202407020706
202407020707
202407020708
202407020709
202407020710
202407020711
202407020712
202401020701
202401020702
202401020703
202401020704
202401020705
202401020706
202401020707
202401020708
202401020709
202401020710
202401020711
202401020712
202404030801
202404030802
202404030803
202404030804
202404030805
202404030806
202404030807
202404030808
202404030809
202404030810
202404030811
202404030812
202407020801
202407020802
202407020803
202407020804
202407020805
202407020806
202407020807
202407020808
202407020809
202407020810
202407020811
202407020812
202401020801
202401020802
202401020803
202401020804
202401020805
202401020806
202401020807
202401020808
202401020809
202401020810
202401020811
202401020812
202406040101
202406040102
202406040103
202406040104
202406040105
202406040106
202406040107
202406040108
202406040109
202406040110
202406040111
202406040112
202407030101
202407030102
202407030103
202407030104
202407030105
202407030106
202407030107
202407030108
202407030109
202407030110
202407030111
202407030112
202406040201
202406040202
202406040203
202406040204
202406040205
202406040206
202406040207
202406040208
202406040209
202406040210
202406040211
202406040212
202407030201
202407030202
202407030203
202407030204
202407030205
202407030206
202407030207
202407030208
202407030209
202407030210
202407030211
202407030212
202406040301
202406040302
202406040303
202406040304
202406040305
202406040306
202406040307
202406040308
202406040309
202406040310
202406040311
202406040312
202407030301
202407030302
202407030303
202407030304
202407030305
202407030306
202407030307
202407030308
202407030309
202407030310
202407030311
202407030312
202406040401
202406040402
202406040403
202406040404
202406040405
202406040406
202406040407
202406040408
202406040409
202406040410
202406040411
202406040412
202407030401
202407030402
202407030403
202407030404
202407030405
202407030406
202407030407
202407030408
202407030409
202407030410
202407030411
202407030412
202406040501
202406040502
202406040503
202406040504
202406040505
202406040506
202406040507
202406040508
202406040509
202406040510
202406040511
202406040512
202407030501
202407030502
202407030503
202407030504
202407030505
202407030506
202407030507
202407030508
202407030509
202407030510
202407030511
202407030512
202406040601
202406040602
202406040603
202406040604
202406040605
202406040606
202406040607
202406040608
202406040609
202406040610
202406040611
202406040612
202407030601
202407030602
202407030603
202407030604
202407030605
202407030606
202407030607
202407030608
202407030609
202407030610
202407030611
202407030612
202406040701
202406040702
202406040703
202406040704
202406040705
202406040706
202406040707
202406040708
202406040709
202406040710
202406040711
202406040712
202407030701
202407030702
202407030703
202407030704
202407030705
202407030706
202407030707
202407030708
202407030709
202407030710
202407030711
202407030712
202406040801
202406040802
202406040803
202406040804
202406040805
202406040806
202406040807
202406040808
202406040809
202406040810
202406040811
202406040812
202407030801
202407030802
202407030803
202407030804
202407030805
202407030806
202407030807
202407030808
202407030809
202407030810
202407030811
202407030812
202406040901
202406040902
202406040903
202406040904
202406040905
202406040906
202406040907
202406040908
202406040909
202406040910
202406040911
202406040912
202407030901
202407030902
202407030903
202407030904
202407030905
202407030906
202407030907
202407030908
202407030909
202407030910
202407030911
202407030912
202405040101
202405040102
202405040103
202405040104
202405040105
202405040106
202405040107
202405040108
202405040109
202405040110
202405040111
202405040112
202408050101
202408050102
202408050103
202408050104
202408050105
202408050106
202408050107
202408050108
202408050109
202408050110
202408050111
202408050112
202404040101
202404040102
202404040103
202404040104
202404040105
202404040106
202404040107
202404040108
202404040109
202404040110
202404040111
202404040112
202405040201
202405040202
202405040203
202405040204
202405040205
202405040206
202405040207
202405040208
202405040209
202405040210
202405040211
202405040212
202408050201
202408050202
202408050203
202408050204
202408050205
202408050206
202408050207
202408050208
202408050209
202408050210
202408050211
202408050212
202404040201
202404040202
202404040203
202404040204
202404040205
202404040206
202404040207
202404040208
202404040209
202404040210
202404040211
202404040212
202408050301
202408050302
202408050303
202408050304
202408050305
202408050306
202408050307
202408050308
202408050309
202408050310
202408050311
202408050312
202404040301
202404040302
202404040303
202404040304
202404040305
202404040306
202404040307
202404040308
202404040309
202404040310
202404040311
202404040312
202405040301
202405040302
202405040303
202405040304
202405040305
202405040306
202405040307
202405040308
202405040309
202405040310
202405040311
202405040312
202408050401
202408050402
202408050403
202408050404
202408050405
202408050406
202408050407
202408050408
202408050409
202408050410
202408050411
202408050412
202405040401
202405040402
202405040403
202405040404
202405040405
202405040406
202405040407
202405040408
202405040409
202405040410
202405040411
202405040412
202404040401
202404040402
202404040403
202404040404
202404040405
202404040406
202404040407
202404040408
202404040409
202404040410
202404040411
202404040412
202405040501
202405040502
202405040503
202405040504
202405040505
202405040506
202405040507
202405040508
202405040509
202405040510
202405040511
202405040512
202408050501
202408050502
202408050503
202408050504
202408050505
202408050506
202408050507
202408050508
202408050509
202408050510
202408050511
202408050512
202404040501
202404040502
202404040503
202404040504
202404040505
202404040506
202404040507
202404040508
202404040509
202404040510
202404040511
202404040512
202405040601
202405040602
202405040603
202405040604
202405040605
202405040606
202405040607
202405040608
202405040609
202405040610
202405040611
202405040612
202408050601
202408050602
202408050603
202408050604
202408050605
202408050606
202408050607
202408050608
202408050609
202408050610
202408050611
202408050612
202404040601
202404040602
202404040603
202404040604
202404040605
202404040606
202404040607
202404040608
202404040609
202404040610
202404040611
202404040612
202405040701
202405040702
202405040703
202405040704
202405040705
202405040706
202405040707
202405040708
202405040709
202405040710
202405040711
202405040712
202408050701
202408050702
202408050703
202408050704
202408050705
202408050706
202408050707
202408050708
202408050709
202408050710
202408050711
202408050712
202404040701
202404040702
202404040703
202404040704
202404040705
202404040706
202404040707
202404040708
202404040709
202404040710
202404040711
202404040712
202405040801
202405040802
202405040803
202405040804
202405040805
202405040806
202405040807
202405040808
202405040809
202405040810
202405040811
202405040812
202408050801
202408050802
202408050803
202408050804
202408050805
202408050806
202408050807
202408050808
202408050809
202408050810
202408050811
202408050812
202404040801
202404040802
202404040803
202404040804
202404040805
202404040806
202404040807
202404040808
202404040809
202404040810
202404040811
202404040812
202405050101
202405050102
202405050103
202405050104
202405050105
202405050106
202405050107
202405050108
202405050109
202405050110
202405050111
202405050112
202408060101
202408060102
202408060103
202408060104
202408060105
202408060106
202408060107
202408060109
202408060110
202408060111
202408060112
202403030101
202403030102
202403030103
202403030104
202403030105
202403030106
202403030107
202403030108
202403030109
202403030110
202403030111
202403030112
202405050201
202405050202
202405050203
202405050204
202405050205
202405050206
202405050207
202405050208
202405050209
202405050210
202405050211
202405050212
202408060201
202408060202
202408060203
202408060204
202408060205
202408060206
202408060207
202408060208
202408060209
202408060210
202408060211
202408060212
202403030201
202403030202
202403030203
202403030204
202403030205
202403030206
202403030207
202403030208
202403030209
202403030210
202403030211
202403030212
202405050301
202405050302
202405050303
202405050304
202405050305
202405050306
202405050307
202405050308
202405050309
202405050310
202405050311
202405050312
202408060301
202408060302
202408060303
202408060304
202408060305
202408060306
202408060307
202408060308
202408060309
202408060310
202408060311
202408060312
202403030301
202403030302
202403030303
202403030304
202403030305
202403030306
202403030307
202403030308
202403030309
202403030310
202403030311
202403030312
202405050401
202405050402
202405050403
202405050404
202405050405
202405050406
202405050407
202405050408
202405050409
202405050410
202405050411
202405050412
202408060401
202408060402
202408060403
202408060404
202408060405
202408060406
202408060407
202408060408
202408060409
202408060410
202408060411
202408060412
202403030401
202403030402
202403030403
202403030404
202403030405
202403030406
202403030407
202403030408
202403030409
202403030410
202403030411
202403030412
202405050501
202405050502
202405050503
202405050504
202405050505
202405050506
202405050507
202405050508
202405050509
202405050510
202405050511
202405050512
202408060501
202408060502
202408060503
202408060504
202408060505
202408060506
202408060507
202408060508
202408060509
202408060510
202408060511
202408060512
202403030501
202403030502
202403030503
202403030504
202403030505
202403030506
202403030507
202403030508
202403030509
202403030510
202403030511
202403030512
202405050601
202405050602
202405050603
202405050604
202405050605
202405050606
202405050607
202405050608
202405050609
202405050610
202405050611
202405050612
202408060601
202408060602
202408060603
202408060604
202408060605
202408060606
202408060607
202408060608
202408060609
202408060610
202408060611
202408060612
202403030601
202403030602
202403030603
202403030604
202403030605
202403030606
202403030607
202403030608
202403030609
202403030610
202403030611
202403030612
202405050701
202405050702
202405050703
202405050704
202405050705
202405050706
202405050707
202405050708
202405050709
202405050710
202405050711
202405050712
202408060701
202408060702
202408060703
202408060704
202408060705
202408060706
202408060707
202408060708
202408060709
202408060710
202408060711
202408060712
202405050801
202405050802
202405050803
202405050804
202405050805
202405050806
202405050807
202405050808
202405050809
202405050810
202405050811
202405050812
202408060801
202408060802
202408060803
202408060804
202408060805
202408060806
202408060807
202408060808
202408060809
202408060810
202408060811
202408060812
202406050101
202406050102
202406050103
202406050104
202406050105
202406050106
202406050107
202406050108
202406050109
202406050110
202406050111
202406050112
202408070101
202408070102
202408070103
202408070104
202408070105
202408070106
202408070107
202408070108
202408070109
202408070110
202408070111
202408070112
202407040101
202407040102
202407040103
202407040104
202407040105
202407040106
202407040107
202407040108
202407040109
202407040110
202407040111
202407040112
202406050201
202406050202
202406050203
202406050204
202406050205
202406050206
202406050207
202406050208
202406050209
202406050210
202406050211
202406050212
202408070201
202408070202
202408070203
202408070204
202408070205
202408070206
202408070207
202408070208
202408070209
202408070210
202408070211
202408070212
202407040201
202407040202
202407040203
202407040204
202407040205
202407040206
202407040207
202407040208
202407040209
202407040210
202407040211
202407040212
202406050301
202406050302
202406050303
202406050304
202406050305
202406050306
202406050307
202406050308
202406050309
202406050310
202406050311
202406050312
202408070301
202408070302
202408070303
202408070304
202408070305
202408070306
202408070307
202408070308
202408070309
202408070310
202408070311
202408070312
202407040301
202407040302
202407040303
202407040304
202407040305
202407040306
202407040307
202407040308
202407040309
202407040310
202407040311
202407040312
202406050401
202406050402
202406050403
202406050404
202406050405
202406050406
202406050407
202406050408
202406050409
202406050410
202406050411
202406050412
202408070401
202408070402
202408070403
202408070404
202408070405
202408070406
202408070407
202408070408
202408070409
202408070410
202408070411
202408070412
202407040401
202407040402
202407040403
202407040404
202407040405
202407040406
202407040407
202407040408
202407040409
202407040410
202407040411
202407040412
202406050501
202406050502
202406050503
202406050504
202406050505
202406050506
202406050507
202406050508
202406050509
202406050510
202406050511
202406050512
202408070501
202408070502
202408070503
202408070504
202408070505
202408070506
202408070507
202408070508
202408070509
202408070510
202408070511
202408070512
202407040501
202407040502
202407040503
202407040504
202407040505
202407040506
202407040507
202407040508
202407040509
202407040510
202407040511
202407040512
202406050601
202406050602
202406050603
202406050604
202406050605
202406050606
202406050607
202406050608
202406050609
202406050610
202406050611
202406050612
202408070601
202408070602
202408070603
202408070604
202408070605
202408070606
202408070607
202408070608
202408070609
202408070610
202408070611
202408070612
202407040601
202407040602
202407040603
202407040604
202407040605
202407040606
202407040607
202407040608
202407040609
202407040610
202407040611
202407040612
202406050701
202406050702
202406050703
202406050704
202406050705
202406050706
202406050707
202406050708
202406050709
202406050710
202406050711
202406050712
202408070701
202408070702
202408070703
202408070704
202408070705
202408070706
202408070707
202408070708
202408070709
202408070710
202408070711
202408070712
202406050801
202406050802
202406050803
202406050804
202406050805
202406050806
202406050807
202406050808
202406050809
202406050810
202406050811
202406050812
202408070801
202408070802
202408070803
202408070804
202408070805
202408070806
202408070807
202408070808
202408070809
202408070810
202408070811
202408070812
202406050901
202406050902
202406050903
202406050904
202406050905
202406050906
202406050907
202406050908
202406050909
202406050910
202406050911
202406050912
202408070901
202408070902
202408070903
202408070904
202408070905
202408070906
202408070907
202408070908
202408070909
202408070910
202408070911
202408070912"""



# 改行で分割してリストに変換
numbers_list = race_id_list2.split("\n")

# 各数字にシングルクォーテーションを追加
race_id_list = [f'{num}' for num in numbers_list]



In [55]:
# raceページのhtmlをスクレイピング
html_paths_race = scraping.scrape_html_race(race_id_list=race_id_list)

100%|██████████████████████████████████████████████████████████████| 252/252 [00:00<00:00, 4323.26it/s]

skipped: 202406050201
skipped: 202406050202
skipped: 202406050203
skipped: 202406050204
skipped: 202406050205
skipped: 202406050206
skipped: 202406050207
skipped: 202406050208
skipped: 202406050209
skipped: 202406050210
skipped: 202406050211
skipped: 202406050212
skipped: 202408070201
skipped: 202408070202
skipped: 202408070203
skipped: 202408070204
skipped: 202408070205
skipped: 202408070206
skipped: 202408070207
skipped: 202408070208
skipped: 202408070209
skipped: 202408070210
skipped: 202408070211
skipped: 202408070212
skipped: 202407040201
skipped: 202407040202
skipped: 202407040203
skipped: 202407040204
skipped: 202407040205
skipped: 202407040206
skipped: 202407040207
skipped: 202407040208
skipped: 202407040209
skipped: 202407040210
skipped: 202407040211
skipped: 202407040212
skipped: 202406050301
skipped: 202406050302
skipped: 202406050303
skipped: 202406050304
skipped: 202406050305
skipped: 202406050306
skipped: 202406050307
skipped: 202406050308
skipped: 202406050309
skipped: 2

In [ ]:
#エラーが起きたレースidについての処理

error_race_ids = ["201805050811"] 


import re
import time
import traceback
from pathlib import Path
from urllib.request import urlopen

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from tqdm.notebook import tqdm
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

from urllib.request import urlopen, Request
from urllib.error import HTTPError




DATA_DIR = Path("..", "data")
HTML_RACE_DIR = DATA_DIR / "html" / "race"
HTML_HORSE_DIR = DATA_DIR / "html" / "horse"
HTML_PED_DIR = DATA_DIR / "html" / "ped"
HTML_LEADING_DIR = DATA_DIR / "html" / "leading"


def error_html_race(
    error_race_ids: list[str], save_dir: Path = HTML_RACE_DIR, skip: bool = True
) -> list[Path]:
    """
    netkeiba.comのraceページのhtmlをスクレイピングしてsave_dirに保存する関数。
    skip=Trueにすると、すでにhtmlが存在する場合はスキップされる。
    逆に上書きしたい場合は、skip=Falseにする。
    スキップされたhtmlのパスは返り値に含まれない。
    """
    updated_html_path_list = []
    save_dir.mkdir(parents=True, exist_ok=True)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}

    for race_id in tqdm(error_race_ids):
        filepath = save_dir / f"{race_id}.bin"
        try:
            url = f"https://db.netkeiba.com/race/{race_id}"
            request = Request(url, headers=headers)
            html = urlopen(request).read()
            time.sleep(1)
            with open(filepath, "wb") as f:
                f.write(html)
            updated_html_path_list.append(filepath)
        except HTTPError as e:
            print(f"Error fetching {race_id}: {e}")

    return updated_html_path_list

error_paths_race = error_html_race(error_race_ids=error_race_ids)

In [ ]:
# 途中で処理が途切れるなどした場合は、直接htmlのファイルパスを取得
html_paths_race = [
    scraping.HTML_RACE_DIR / f"{race_id}.bin" for race_id in race_id_list
]

In [56]:
html_paths_race = [
    scraping.HTML_RACE_DIR / f"{race_id}.bin" for race_id in race_id_list
]

In [57]:
html_paths_race

[PosixPath('../data/html/race/202406050201.bin'),
 PosixPath('../data/html/race/202406050202.bin'),
 PosixPath('../data/html/race/202406050203.bin'),
 PosixPath('../data/html/race/202406050204.bin'),
 PosixPath('../data/html/race/202406050205.bin'),
 PosixPath('../data/html/race/202406050206.bin'),
 PosixPath('../data/html/race/202406050207.bin'),
 PosixPath('../data/html/race/202406050208.bin'),
 PosixPath('../data/html/race/202406050209.bin'),
 PosixPath('../data/html/race/202406050210.bin'),
 PosixPath('../data/html/race/202406050211.bin'),
 PosixPath('../data/html/race/202406050212.bin'),
 PosixPath('../data/html/race/202408070201.bin'),
 PosixPath('../data/html/race/202408070202.bin'),
 PosixPath('../data/html/race/202408070203.bin'),
 PosixPath('../data/html/race/202408070204.bin'),
 PosixPath('../data/html/race/202408070205.bin'),
 PosixPath('../data/html/race/202408070206.bin'),
 PosixPath('../data/html/race/202408070207.bin'),
 PosixPath('../data/html/race/202408070208.bin'),


In [58]:
results = create_rawdf.create_results(html_path_list=html_paths_race)

  0%|          | 0/252 [00:00<?, ?it/s]

table not found at 202408070305
table not found at 202408070504
table not found at 202406050604
table not found at 202408070602
table not found at 202406050710
table not found at 202408070805
table not found at 202408070904


In [146]:
results

,race_id,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,ﾀｲﾑ指数,通過,上り,単勝,人気,馬体重,調教ﾀｲﾑ,厩舎ｺﾒﾝﾄ,備考,調教師,馬主,賞金(万円),horse_id,jockey_id,trainer_id,owner_id
0,202406050201,1,3,6,レーヴレジェンド,牡2,56.0,菊沢一樹,1:12.6,NaN,**,1-1,37.9,3.7,2.0,494(+8),NaN,NaN,NaN,[東] 金成貴史,細谷典幸,550.0,2022101409,01161,01132,443007
1,202406050201,2,6,12,サードウインド,牝2,55.0,丹内祐次,1:12.8,1.1/4,**,4-4,37.6,2.1,1.0,458(+4),NaN,NaN,NaN,[東] 西田雄一,青芝商事,220.0,2022104546,01091,01187,831800
2,202406050201,3,7,13,ダイチラポール,牡2,53.0,長浜鴻緒,1:13.2,2.1/2,**,2-2,38.2,6.0,3.0,466(+2),NaN,NaN,NaN,[東] 蛯名利弘,服部英利,140.0,2022100076,01214,01131,921031
3,202406050201,4,6,11,ダングラール,牡2,56.0,マーカン,1:13.4,1.1/4,**,2-2,38.5,8.0,4.0,462(+2),NaN,NaN,NaN,[東] 田中博康,橋元勇氣,83.0,2022105288,05626,01162,943030
4,202406050201,5,2,4,ミルトオーケストラ,牡2,56.0,石橋脩,1:14.0,3.1/2,**,8-6,38.6,38.7,7.0,448(+2),NaN,NaN,NaN,[東] 武市康男,永山勝敏,55.0,2022100987,01077,01089,695033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,202408070912,12,2,3,ジュノー,セ6,58.0,川須栄彦,1:09.6,3/4,**,16-16,33.8,36.6,8.0,536(+6),NaN,NaN,NaN,[西] 音無秀孝,金子真人ホールディングス,NaN,2018104757,01128,01002,708800
12,202408070912,13,5,10,ナリタローゼ,牝6,56.0,角田大和,1:09.6,アタマ,**,9-9,34.6,65.4,11.0,494(0),NaN,NaN,NaN,[西] 高橋亮,オースミ,NaN,2018104559,01186,01136,712800
13,202408070912,14,5,9,ピンクマクフィー,牝5,56.0,斎藤新,1:09.7,クビ,**,5-5,35.0,26.1,7.0,448(+8),NaN,NaN,NaN,[西] 四位洋文,スリーエイチレーシング,NaN,2019101626,01178,01184,072803
14,202408070912,15,1,2,コナブラック,牡5,58.0,亀田温心,1:09.8,3/4,**,11-13,34.5,81.1,12.0,524(+4),NaN,NaN,NaN,[西] 清水久詞,サンデーレーシング,NaN,2019105191,01176,01110,226800


In [59]:
#エラーが起きたidについての処理
import re


# race_id_list2 は改行で区切られた文字列
mess = """table not found at 202408070305
table not found at 202408070504
table not found at 202406050604
table not found at 202408070602
table not found at 202406050710
table not found at 202408070805
table not found at 202408070904"""

# 改行で分割してリストに変換
mess2 = mess.split("\n")

# 各数字にシングルクォーテーションを追加してリストに格納
messages = [f"{num}" for num in mess2]

# 正規表現を使って数字部分を抽出
missing_tables = [re.search(r'\d{12}', message).group() for message in messages]


In [173]:
missing_tables

['202410020704',
 '202410020705',
 '202410020804',
 '202404010604',
 '202404010605',
 '202405030211',
 '202408040202',
 '202403020501',
 '202403020504',
 '202410030601',
 '202410030709',
 '202403020808',
 '202407020105',
 '202407020501',
 '202407020506',
 '202404030601',
 '202404030701',
 '202404030704',
 '202407020801',
 '202401020809',
 '202407030103',
 '202407030310',
 '202406040407',
 '202406040503',
 '202407030501',
 '202407030601',
 '202407030604',
 '202406040704',
 '202407030701',
 '202406040906',
 '202405040108',
 '202404040101',
 '202405040203',
 '202404040204',
 '202408050304',
 '202408050309',
 '202404040304',
 '202408050402',
 '202408050404',
 '202408050412',
 '202404040406',
 '202408050601',
 '202408050605',
 '202404040602',
 '202408050707',
 '202408050712',
 '202404040701',
 '202408050812',
 '202403030104',
 '202408060202',
 '202403030202',
 '202403030203',
 '202403030208',
 '202405050412',
 '202408060402',
 '202403030411',
 '202408060506',
 '202408060604',
 '202408060606

In [60]:
#エラーが起きたidについての処理2
import re
from pathlib import Path
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import numpy as np

DATA_DIR = Path("..", "data")
RAWDF_DIR = DATA_DIR / "rawdf"

# 例: 取得時にエラーが発生したrace_idのリスト
error_race_ids = missing_tables

dfs = {}

for error_race_id in tqdm(error_race_ids):
    try:
        # HTMLファイルのパスを指定
        html_path = DATA_DIR / "html" / "race" / f"{error_race_id}.bin"  
        with open(html_path, "rb") as f:
            html = (
                f.read()
                .replace(b"<diary_snap_cut>", b"")
                .replace(b"</diary_snap_cut>", b"")
            )
        
        soup = BeautifulSoup(html, "lxml").find("table", class_="race_table_01 nk_tb_common")
        
        # テーブルが見つからない場合はスキップ
        if soup is None:
            print(f"table not found for race_id: {error_race_id}")
            continue
        
        # HTML から DataFrame を抽出
        df = pd.read_html(html)[0]

        # horse_id, jockey_id, trainer_id, owner_id の抽出処理を関数化
        def extract_ids(soup, pattern, id_length):
            id_list = []
            a_list = soup.find_all("a", href=re.compile(pattern))
            for a in a_list:
                match = re.findall(rf"\d{{{id_length}}}", a["href"])
                id_list.append(match[0] if match else np.nan)
            
            # id_listの長さが df に満たない場合は NaN で埋める
            if len(id_list) < len(df):
                id_list.extend([np.nan] * (len(df) - len(id_list)))
            elif len(id_list) > len(df):
                id_list = id_list[:len(df)]
            return id_list
        
        df["horse_id"] = extract_ids(soup, r"^/horse/", 10)
        df["jockey_id"] = extract_ids(soup, r"^/jockey/", 5)
        df["trainer_id"] = extract_ids(soup, r"^/trainer/", 5)
        df["owner_id"] = extract_ids(soup, r"^/owner/", 6)

        # 最初の列にrace_idを挿入
        df.insert(0, "race_id", error_race_id)
        
        # レースIDごとに DataFrame を保存
        dfs[error_race_id] = df
    
    except IndexError as e:
        print(f"IndexError at {error_race_id}: {e}")
        continue
    except Exception as e:
        print(f"An error occurred for race_id {error_race_id}: {e}")
        continue


# すべての DataFrame を結合
concat_df = pd.concat(dfs.values())
concat_df.columns = concat_df.columns.str.replace(" ", "")
results = pd.concat([results, concat_df], ignore_index=True)

                    
# CSVを更新
def update_rawdf(
    new_df: pd.DataFrame,
    key: str,
    save_filename: str,
    save_dir: Path = RAWDF_DIR,
) -> None:
    """
    既存のrawdfに新しいデータを追加して保存する関数。
    """
    save_dir.mkdir(parents=True, exist_ok=True)
    if (save_dir / save_filename).exists():
        old_df = pd.read_csv(save_dir / save_filename, sep="\t", dtype={f"{key}": str})
        new_df[key] = new_df[key].astype(str)
        df = pd.concat([old_df[~old_df[key].isin(new_df[key])], new_df])
        df.to_csv(save_dir / save_filename, sep="\t", index=False)
    else:
        new_df.to_csv(save_dir / save_filename, sep="\t", index=False)


update_rawdf(
    concat_df,
    key="race_id",
    save_filename="results.csv",
    save_dir=RAWDF_DIR,
)

  0%|          | 0/7 [00:00<?, ?it/s]

IndexError at 202408070504: list index out of range
IndexError at 202406050604: list index out of range
IndexError at 202406050710: list index out of range
IndexError at 202408070805: list index out of range
IndexError at 202408070904: list index out of range


In [175]:
results

,race_id,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,ﾀｲﾑ指数,通過,上り,単勝,人気,馬体重,調教ﾀｲﾑ,厩舎ｺﾒﾝﾄ,備考,調教師,馬主,賞金(万円),horse_id,jockey_id,trainer_id,owner_id
0,202406010101,1,7,14,サンディブロンド,牝3,55.0,戸崎圭太,1:12.6,NaN,**,2-2,38.5,4.2,3.0,466(-12),NaN,NaN,NaN,[東] 田中博康,サンデーレーシング,550.0,2021105821,05386,01162,226800
1,202406010101,2,4,7,マイネルディレクト,牡3,57.0,丹内祐次,1:12.8,1,**,5-4,38.1,8.4,5.0,496(+6),NaN,NaN,NaN,[東] 高橋裕,サラブレッドクラブ・ラフィアン,220.0,2021106805,01091,00412,546800
2,202406010101,3,8,16,セイウンチドリ,牝3,55.0,三浦皇成,1:13.2,2.1/2,**,5-4,38.6,15.3,6.0,446(-6),NaN,NaN,NaN,[東] 村田一誠,西山茂行,140.0,2021102685,01122,01190,897009
3,202406010101,4,2,4,グリントリッター,セ3,57.0,石川裕紀,1:13.2,クビ,**,4-4,38.7,3.9,2.0,420(+2),NaN,NaN,NaN,[東] 深山雅史,ウエスト．フォレスト．ステイブル,83.0,2021100328,01150,01174,368803
4,202406010101,5,6,12,マニーブルース,牡3,57.0,吉田豊,1:13.3,1,**,8-7,38.1,50.9,9.0,458(+4),NaN,NaN,NaN,[東] 小手川準,サンライズ,55.0,2021102752,00733,01171,866800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46989,202408070602,12,2,4,マッハレイジング,牡2,56.0,岩田望来,1:29.7,1.1/4,**,3-5,41.1,64.9,8.0,474(-8),NaN,NaN,NaN,[西] 小崎憲,三浦勝仁,NaN,2022106597,01174,01085,810033
46990,202408070602,13,2,3,コパノジェームス,牡2,56.0,小崎綾也,1:30.1,2.1/2,**,12-11,40.4,557.2,15.0,504(+4),NaN,NaN,NaN,[西] 村山明,小林祥晃,NaN,2022100287,01152,01107,966007
46991,202408070602,14,3,5,レイリースピリット,牡2,54.0,吉村誠之,1:30.4,1.3/4,**,8-9,41.1,233.0,13.0,422(+4),NaN,NaN,NaN,[西] 浜田多実,山田裕二,NaN,2022103944,01216,01138,NaN
46992,202408070602,15,5,10,ナムラブルータス,牡2,55.0,田口貫太,1:31.9,9,**,16-16,40.1,130.6,10.0,482(+8),NaN,NaN,NaN,[西] 大橋勇樹,奈村睦弘,NaN,2022100144,01208,01065,972002


In [125]:
results = pd.read_csv(RAWDF_DIR / "results.csv", sep="\t")

In [61]:
# 重複チェック
results[["race_id", "horse_id"]].duplicated().sum()

np.int64(0)

In [62]:
race_info = create_rawdf.create_race_info(html_path_list=html_paths_race)

  0%|          | 0/252 [00:00<?, ?it/s]

In [127]:
#レース情報テーブルの作成
"""
*注意・create_rawdf.create_race_infoは２度コードを実行すると、return_table.csvに列がどんどん増えていくため、一度の実行にすること
エラーが起きている場合、一度return_table.csvを削除してから行うこと
欠損などで２度実行したい場合は、一度、csvファイルを読み込み、列をまとめてから実行すること
最終状態はまとめた状態にすること
"""
#一回一回、csvファイルを消さないと空欄の列が追加される
return_tables = create_rawdf.create_return_tables(html_path_list=html_paths_race)

  0%|          | 0/252 [00:00<?, ?it/s]

table not found at 202408070503
table not found at 202406050603
table not found at 202406050709
table not found at 202408070804
table not found at 202408070903


In [128]:
#エラー処理
log_text = """
table not found at 202408070503
table not found at 202406050603
table not found at 202406050709
table not found at 202408070804
table not found at 202408070903
"""
import re
race_ids = sorted(set(re.findall(r'table not found at (\d+)', log_text)))
html_paths_race_error = [HTML_RACE_DIR / f"{race_id}.bin" for race_id in race_ids]



In [129]:
#エラー処理続き
#一回一回、csvファイルを消さないと空欄の列が追加される
return_tables3 = create_rawdf.create_return_tables(html_path_list=html_paths_race_error)

  0%|          | 0/5 [00:00<?, ?it/s]

/Users/gotoumasafumi/Downloads/GitHub/horse_racing/common/src/create_rawdf.py:614: DtypeWarning: Columns (5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  old_df = pd.read_csv(save_dir / save_filename, sep="\t", dtype={f"{key}": str})


In [130]:
#列が二重になった場合の処理
#欠損処理をした場合、列が二重になるためここで修正
DATA_DIR = Path("..", "data")
RAWDF_DIR = DATA_DIR / "rawdf"
df = pd.read_csv(RAWDF_DIR/ "return_tables.csv", sep="\t")
# 欠損値を埋める処理
for col in ['0', '1', '2', '3']:  # 列名を文字列として扱う
    col1 = f"{col}.1"  # '0.1', '1.1', '2.1', '3.1'
    df[col] = df[col].combine_first(df[col1])

# 不要な列（0.1, 1.1, 2.1, 3.1）を削除
df.drop(columns=['0.1', '1.1', '2.1', '3.1'], inplace=True)

# 必要なら保存
df.to_csv(RAWDF_DIR/"return_tables.csv", index=False, sep="\t")

/var/folders/43/k8vtkf1x6js1yc8cj0ml9g2h0000gn/T/ipykernel_39425/197898190.py:5: DtypeWarning: Columns (5,6,7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(RAWDF_DIR/ "return_tables.csv", sep="\t")


In [177]:
#馬の過去成績をスクレイピング
horse_id_list = results["horse_id"].unique()
len(horse_id_list)

11781

In [188]:
horse_id_list 

array(['2021105821', '2021106805', '2021102685', ..., '2022101006',
       '2022100885', '2022105471'], shape=(11781,), dtype=object)

In [190]:
html_paths_horse = scraping.scrape_html_horse(
    horse_id_list=horse_id_list, skip=False
)

  0%|                                       | 5/11781 [00:08<5:54:39,  1.81s/it]

An error occurred for horse_id 2021100402: HTTP Error 400: Bad Request


  0%|                                      | 10/11781 [00:13<3:49:45,  1.17s/it]

An error occurred for horse_id 2021102792: HTTP Error 400: Bad Request
An error occurred for horse_id 2021106538: HTTP Error 400: Bad Request


  0%|                                      | 28/11781 [00:37<3:13:00,  1.01it/s]

...

An error occurred for horse_id 2022102115: HTTP Error 400: Bad Request
An error occurred for horse_id 2022106123: HTTP Error 400: Bad Request
An error occurred for horse_id 2022100169: HTTP Error 400: Bad Request


100%|██████████████████████████████████▉| 11778/11781 [1:35:19<00:00, 13.70it/s]

An error occurred for horse_id 2022103830: HTTP Error 400: Bad Request
An error occurred for horse_id 2022103944: HTTP Error 400: Bad Request
An error occurred for horse_id 2022106591: HTTP Error 400: Bad Request
An error occurred for horse_id 2022101006: HTTP Error 400: Bad Request


100%|███████████████████████████████████| 11781/11781 [1:35:20<00:00,  2.06it/s]

An error occurred for horse_id 2022100885: HTTP Error 400: Bad Request
An error occurred for horse_id 2022105471: HTTP Error 400: Bad Request

Errors occurred for the following horse_ids:
['2021100402', '2021102792', '2021106538', '2021106559', '2021100614', '2021107164', '2021104570', '2021103100', '2021101356', '2021103169', '2020103923', '2019106321', '2021106372', '2016103409', '2019100844', '2018104895', '2019100668', '2021105871', '2017104756', '2017101159', '2019105188', '2017101877', '2021102150', '2021100741', '2021107295', '2021104151', '2021105141', '2021105438', '2021101426', '2021105277', '2021109114', '2021106395', '2020105954', '2020103123', '2020103074', '2019103726', '2019103027', '2018106560', '2018105187', '2018103499', '2019101507', '2018104910', '2020103661', '2020103277', '2018105020', '2019105252', '2019100965', '2020100678', '2020104224', '2021104362', '2021109094', '2021101306', '2021101204', '2021100005', '2021106213', '2021104569', '2021104727', '2021101168',

In [141]:
# 途中で処理が途切れるなどした場合は、直接htmlのファイルパスを取得
html_paths_horse = [
    scraping.HTML_HORSE_DIR / f"{horse_id}.bin" for horse_id in horse_id_list
]

In [142]:
html_paths_horse = list(scraping.HTML_HORSE_DIR.glob("*.bin"))

In [192]:
# 馬の過去成績テーブルの作成
horse_results = create_rawdf.create_horse_results(html_paths_horse)

  0%|          | 0/3582 [00:00<?, ?it/s]

In [179]:
#エラーが起きたbinファイルをもう一度取得し、再度作成する
log_text_horse = """
table not found at 2022110143
table not found at 2022110145
table not found at 2022110151
table not found at 2022110152
table not found at 2022110153
"""

horse_ids = sorted(set(re.findall(r'table not found at (\d+)', log_text_horse)))

html_paths_horse_error = scraping.scrape_html_horse(
    horse_id_list=horse_ids, skip=False
)
#上書きしたいのでFalse

['2022110143', '2022110145', '2022110151', '2022110152', '2022110153']

In [ ]:
html_paths_horse_error = [HTML_HORSE_DIR / f"{horse_id}.bin" for horse_id in horse_ids]

In [ ]:
horse_results3 = create_rawdf.create_horse_results(html_paths_horse_error)

In [102]:
#エラー処理、rank_1racesノーデータ
error_horse = [2013104922, 2012102111, 2013102667, 2013103000, 2013103225, 2013105080, 2013104156, 2013104851, 2013104067, 2013104168, 2013101047, 2013103589, 2013104825, 2013103865, 2013102726, 2013104155, 2013102892, 2010190006, 2013100877, 2012105460, 2014104434, 2014104429, 2013104823, 2013102706, 2014101055, 2014101782, 2014102055, 2014104335, 2014100427, 2013100446, 2014100465, 2014101483, 2014103307, 2014101559, 2014109117, 2014106369, 2013103940, 2014103604, 2014101893, 2014105200, 2014104277, 2014102709, 2013106459, 2012103911, 2013110132, 2014101961, 2014101850, 2012109107, 2014101699, 2014104300, 2014104078, 2012190003, 2014102046, 2014101019, 2013102669, 2014105315, 2014106466, 2014103934, 2014104911, 2014101725, 2013110131, 2014104142, 2014106387, 2014103991, 2014106396, 2014101434, 2014104932, 2014104582, 2014103016, 2011100019, 2014101254, 2013102309, 2014100801, 2011190004, 2014104011, 2015102248, 2014103890, 2014102925, 2014103856, 2013101112, 2015101465, 2015104624, 2015100129, 2015103679, 2015101899, 2014101785, 2014100706, 2015100887, 2015101468, 2015103255, 2015100664, 2015101952, 2015100993, 2014100662, 2015109104, 2015109102, 2014104202, 2011100189, 2014102468, 2014105213, 2014101408, 2015106125, 2015102341, 2015102113, 2015103800, 2015101086, 2014101674, 2015101106, 2011190005, 2015100072, 2015103959, 2015101459, 2014101507, 2013190002, 2012190004, 2011190006, 2015100080, 2015100316, 2015101103, 2015102805, 2015103214, 2015100313, 2015100660, 2014105252, 2015105501, 2015102184, 2014101371, 2015105578, 2015105585, 2015101953, 2015101105, 2015105684, 2015100910, 2015101044, 2015102783, 2014101708, 2015103972, 2015100150, 2015109059, 2015106271, 2015105849, 2015102098, 2013103350, 2014104830, 2012190005, 2011190007, 2015105398, 2015106502, 2015101831, 2016105439, 2016105669, 2015101946, 2016103387, 2016106387, 2016106082, 2016101433, 2016105508, 2016102640, 2016103958, 2016105718, 2016103825, 2016102960, 2016106473, 2016100394, 2015100966, 2015104115, 2015103588, 2014103705, 2016102275, 2016104043, 2016102966, 2016105191, 2016101826, 2016101757, 2016103900, 2016103779, 2016101117, 2013190003, 2014190001, 2016102564, 2016103049, 2013103629, 2016101195, 2016102847, 2016103017, 2015100106, 2016103578, 2016101390, 2016100893, 2016105664, 2016101708, 2016101135, 2016103179, 2016104103, 2016101971, 2016102736, 2016103423, 2016104192, 2016101815, 2015100061, 2015103884, 2016103741, 2017103509, 2017102160, 2017100161, 2015102718, 2014103479, 2014104623, 2017100315, 2017100098, 2017106483, 2017103017, 2017101519, 2017102126, 2016101009, 2017102195, 2017100320, 2017105883, 2017103403, 2017101109, 2017104539, 2017106043, 2017109131, 2017102016, 2017102186, 2017101506, 2017101509, 2017100492, 2017102697, 2017103340, 2017106690, 2016100981, 2015106351, 2016102967, 2017100297, 2017103152, 2015105971, 2016104785, 2017106497, 2014100235, 2017100323, 2017103195, 2017100495, 2017102562, 2015102904, 2017101611, 2014102775, 2017104112, 2017110165, 2017106486, 2017100498, 2017106481, 2017104467, 2017105745, 2016100930, 2017102820, 2015110058, 2016100945, 2017110058, 2017101695, 2018103851, 2016100503, 2018102961, 2018101319, 2018106228, 2018105796, 2018104021, 2017103090, 2018105961, 2018102733, 2018101935, 2018104233, 2018103678, 2018106270, 2018102399, 2018109122, 2018109169, 2018103618, 2018100520, 2018100548, 2018100753, 2018105726, 2018102735, 2018103231, 2017101060, 2018104545, 2018106358, 2018105978, 2017105803, 2018100342, 2017103367, 2018102309, 2018104523, 2018100353, 2018100141, 2018104117, 2013190004, 2018105595, 2018104028, 2018105644, 2016102502, 2017104319, 2017100908, 2018102366, 2018106653, 2018102819, 2018105750, 2018100331, 2018104127, 2018105956, 2017106044, 2017102070, 2018105636, 2017106475, 2017101741, 2017106224, 2016102002, 2015103718, 2017104438, 2019105982, 2018109009, 2019100295, 2019110102, 2019102106, 2019101340, 2019101137, 2019103624, 2019100270, 2019100455, 2019109134, 2019109116, 2018102344, 2019105587, 2019102183, 2019101859, 2018100479, 2018104674, 2017106419, 2019109090, 2019106957, 2016190001, 2016190002, 2016190003, 2018103507, 2018105654, 2018100526, 2019100886, 2017100391, 2018103898, 2019103469, 2019103451, 2019103439, 2017100216, 2019100123, 2019106053, 2018105014, 2018101468, 2019106147, 2019106757, 2018100116, 2018101321, 2019104202, 2019100721, 2020100873, 2020105729, 2020102411, 2020104280, 2019105765, 2020107146, 2020106446, 2018106578, 2020100686, 2020109061, 2020103834, 2020100101, 2020109140, 2020109149, 2020109133, 2020109156, 2020109139, 2019104175, 2020102493, 2019100398, 2020100248, 2020102412, 2020104873, 2020101532, 2020105574, 2020101320, 2018103515, 2018106559, 2019104307, 2020104882, 2019190001, 2019190002, 2019190004, 2019190003, 2020100654, 2020105042, 2020105701, 2020106959, 2020102352, 2020102319, 2020106453, 2019105183, 2020100435, 2020100304, 2020105288, 2020100121, 2020102123, 2020104731, 2020109003, 2019101529, 2020100443, 2019100268, 2019106183, 2020101755, 2019106611, 2020102274, 2020102431, 2021101069, 2021102532, 2021104666, 2021104368, 2021104027, 2021109009, 2021102388, 2021102868, 2021102296, 2021100444, 2021109161, 2021109174, 2021109130, 2021109158, 2021103695, 2021109022, 2021106264, 2021106510, 2021103761, 2021101955, 2020102387, 2020105737, 2021102332, 2021101916, 2020103338, 2021107138, 2021106372, 2021105203, 2021100102, 2017190002, 2021102155, 2021105204, 2020106249, 2020103404, 2021100121, 2021100384, 2021106730, 2019103013, 2021100447, 2021103136, 2020104285, 2018105352, 2021104167, 2021101386, 2021105237, 2021101385, 2020106751, 2020103867, 2019102665, 2021102904, 2021102224, 2018190001, 2021105056, 2021107173, 2021105864, 2021107180, 2021102224, 2021103557, 2021105864, 2021100143, 2020101242, 2021102224, 2020101115, 2018190002, 2018190003, 2021100143, 2020110050, 2021100055, 2021101134, 2021107180, 2021105102, 2020102238, 2021102904, 2021107173, 2021103557, 2022105718, 2021101619, 2020102775, 2022109171, 2022109107, 2022109155, 2021107173, 2021103259, 2021105056, 2022110015, 2022106874, 2022100789, 2021100867, 2022100161, 2022102614, 2022104509, 2022101909, 2021103557, 2021101488, 2022103220, 2022109024, 2022106498, 2022103355, 2022106104, 2022100855, 2021101134, 2022105163, 2022104771, 2021100143, 2022104772, 2022104720, 2022105830, 2022100161, 2022102794, 2022102086, 2022107209, 2022107304, 2021107180, 2021105864, 2022102849, 2022106394, 2022104028, 2021105102, 2021100143, 2021110090, 2021105056, 2022110108, 2022106874, 2022103995, 2022105590, 2021107180, 2021103557, 2018190004, 2022104751, 2022104898, 2022102617, 2021110090, 2022105590, 2022105718, 2022110014, 2022101745, 2021103557, 2021104931, 2022100286, 2021101134, 2021107173, 2021105574, 2022104994, 2022101197, 2022105402, 2022104812, 2021105680, 2022101732, 2022106394, 2022104929, 2022102849, 2021101487, 2022105421, 2022105291, 2022105891, 2022102148, 2021107173, 2020106694, 2022104771, 2022105102, 2022104720, 2022105951, 2021105102, 2021101487, 2020190002, 2021105864, 2018190005, 2022103995, 2022102217, 2022100874, 2021105574, 2020190005, 2020190003, 2020190004, 2022104832, 2022103288, 2021102904, 2020103864, 2021100143, 2021105214, 2022110012, 2022100982]


In [103]:
# 途中で処理が途切れるなどした場合は、直接htmlのファイルパスを取得
horse_results_none_pre = [
    scraping.HTML_HORSE_DIR / f"{n}.bin" for n in error_horse
]

In [104]:
horse_results_none = create_rawdf.create_horse_results(horse_results_none_pre)

  0%|          | 0/602 [00:00<?, ?it/s]

2013104922 は新馬のレビューがあるためスキップ
2012102111 は新馬のレビューがあるためスキップ
2013102667 は新馬のレビューがあるためスキップ
2013103000 は新馬のレビューがあるためスキップ
2013103225 は新馬のレビューがあるためスキップ
2013105080 は新馬のレビューがあるためスキップ
2013104156 は新馬のレビューがあるためスキップ
2013104851 は新馬のレビューがあるためスキップ
2013104067 は新馬のレビューがあるためスキップ
2013104168 は新馬のレビューがあるためスキップ
2013101047 は新馬のレビューがあるためスキップ
2013103589 は新馬のレビューがあるためスキップ
2013104825 は新馬のレビューがあるためスキップ
2013103865 は新馬のレビューがあるためスキップ
2013102726 は新馬のレビューがあるためスキップ
2013104155 は新馬のレビューがあるためスキップ
2013102892 は新馬のレビューがあるためスキップ
2010190006 は新馬のレビューがあるためスキップ
2013100877 は新馬のレビューがあるためスキップ
2012105460 は新馬のレビューがあるためスキップ
2014104434 は新馬のレビューがあるためスキップ
2014104429 は新馬のレビューがあるためスキップ
2013104823 は新馬のレビューがあるためスキップ
2013102706 は新馬のレビューがあるためスキップ
2014101055 は新馬のレビューがあるためスキップ
2014101782 は新馬のレビューがあるためスキップ
2014102055 は新馬のレビューがあるためスキップ
2014104335 は新馬のレビューがあるためスキップ
2014100427 は新馬のレビューがあるためスキップ
2013100446 は新馬のレビューがあるためスキップ
2014100465 は新馬のレビューがあるためスキップ
2014101483 は新馬のレビューがあるためスキップ
2014103307 は新馬のレビューがあるためスキップ
2014101559 は新馬のレビューがあるためスキップ
2014109117 は新馬

In [25]:
#血統情報の取得
if "results" not in locals():
    results = pd.read_csv(create_rawdf.RAWDF_DIR / "results.csv", sep="\t")
horse_id_list = results["horse_id"].unique()

In [29]:
html_paths_ped = scraping.scrape_html_ped(
    horse_id_list=horse_id_list,
    save_dir=Path(HTML_PED_DIR)
)


 58%|████████████████████▎              | 6840/11781 [00:00<00:00, 68397.95it/s]

skipped: 2021105821
skipped: 2021106805
skipped: 2021102685
skipped: 2021100328
skipped: 2021102752
skipped: 2021100402
skipped: 2021104782
skipped: 2021103105
skipped: 2021101169
skipped: 2021102792
skipped: 2021106538
skipped: 2021103974
skipped: 2021101587
skipped: 2021106007
skipped: 2021100211
skipped: 2021103189
skipped: 2021107212
skipped: 2021106874
skipped: 2021100469
skipped: 2021104073
skipped: 2021104850
skipped: 2021102342
skipped: 2021100627
skipped: 2021101591
skipped: 2021103758
skipped: 2021103174
skipped: 2021102925
skipped: 2021106559
skipped: 2021106224
skipped: 2021100614
skipped: 2021102394
skipped: 2021104837
skipped: 2021105032
skipped: 2021103744
skipped: 2021103326
skipped: 2021100637
skipped: 2021104487
skipped: 2021103317
skipped: 2021106995
skipped: 2021107164
skipped: 2021103511
skipped: 2021100608
skipped: 2021106365
skipped: 2021104570
skipped: 2021104546
skipped: 2021101881
skipped: 2021103100
skipped: 2021102025
skipped: 2021101356
skipped: 2021103109


 95%|██████████████████████████████████▎ | 11227/11781 [00:14<00:00, 635.51it/s]

skipped: 2022105716
skipped: 2022104404
skipped: 2022103862
skipped: 2022100154
skipped: 2020105302
skipped: 2019103672
skipped: 2020102772
skipped: 2018110015
skipped: 2019104564
skipped: 2019104678
skipped: 2020103400
skipped: 2020103370
skipped: 2020107121
skipped: 2021105783
skipped: 2022105261
skipped: 2022102920
skipped: 2022102393
skipped: 2022104552
skipped: 2022102980


 96%|██████████████████████████████████▍ | 11258/11781 [00:20<00:01, 389.88it/s]

...

 98%|████████████████████████████████████ | 11501/11781 [01:34<00:16, 17.16it/s]

skipped: 2022106925
skipped: 2022103941
skipped: 2022103255
skipped: 2022103306
skipped: 2022105193
skipped: 2022104125
skipped: 2022101862
skipped: 2022107133


 98%|████████████████████████████████████▏| 11510/11781 [01:34<00:15, 17.12it/s]

skipped: 2022103123
skipped: 2022104513
skipped: 2022104732
skipped: 2022100542
skipped: 2022103354
skipped: 2022100222
skipped: 2020104725
skipped: 2019101406
skipped: 2019105111
skipped: 2022101023
skipped: 2022104767
skipped: 2022106228
skipped: 2022104441
skipped: 2022105014
skipped: 2022102087
skipped: 2022105454
skipped: 2022104773
skipped: 2022100332
skipped: 2022106888
skipped: 2022102022
skipped: 2022104350
skipped: 2022106962
skipped: 2022106116
skipped: 2022101789
skipped: 2022106794
skipped: 2022103169
skipped: 2022100394
skipped: 2022110126
skipped: 2022103538
skipped: 2022102063
skipped: 2022100912
skipped: 2022101620
skipped: 2022100927
skipped: 2022105283
skipped: 2022109090
skipped: 2022103849
skipped: 2022105311
skipped: 2022103303
skipped: 2022101851
skipped: 2022103301
skipped: 2022100769
skipped: 2020106979
skipped: 2022103080
skipped: 2022104648
skipped: 2022100725
skipped: 2022101896
skipped: 2022105216
skipped: 2022105385
skipped: 2022105330
skipped: 2022110101


100%|████████████████████████████████████| 11781/11781 [01:53<00:00, 104.05it/s]


In [ ]:
#エラーが起きた場合の処理
import csv
from pathlib import Path

# CSVファイルのパスを指定
error_log_path = Path('../data/html/ped/error_horse_ids.csv')

# CSVファイルからhorse_idをリストに読み込み
with open(error_log_path, "r", newline="") as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # ヘッダーをスキップ
    error_horse_ids_list = [row[0] for row in reader]

# 読み込んだエラーのhorse_idを表
error_horse_ids_list = list(map(int, error_horse_ids_list))
error_horse_ids_list = np.array(error_horse_ids_list)
error_html_paths_ped = scraping.scrape_html_ped(
    horse_id_list=error_horse_ids_list,
    save_dir=Path(HTML_PED_DIR)
)

In [ ]:
# 途中で処理が途切れるなどした場合は、直接htmlのファイルパスを取得
html_paths_ped = [
    scraping.HTML_PED_DIR / f"{horse_id}.bin" for horse_id in horse_id_list
]

In [32]:
# 血統テーブルの作成
peds = create_rawdf.create_peds(html_paths_ped)

  0%|          | 0/2 [00:00<?, ?it/s]

TypeError: expected str, bytes or os.PathLike object, not list

In [33]:
html_paths_ped

([PosixPath('../data/html/ped/2022104490.bin'),
  PosixPath('../data/html/ped/2022102352.bin'),
  PosixPath('../data/html/ped/2022102031.bin'),
  PosixPath('../data/html/ped/2022101820.bin'),
  PosixPath('../data/html/ped/2022102093.bin'),
  PosixPath('../data/html/ped/2022103572.bin'),
  PosixPath('../data/html/ped/2022103030.bin'),
  PosixPath('../data/html/ped/2022101287.bin'),
  PosixPath('../data/html/ped/2022104592.bin'),
  PosixPath('../data/html/ped/2022101718.bin'),
  PosixPath('../data/html/ped/2022105597.bin'),
  PosixPath('../data/html/ped/2022107242.bin'),
  PosixPath('../data/html/ped/2022104695.bin'),
  PosixPath('../data/html/ped/2022105852.bin'),
  PosixPath('../data/html/ped/2022105681.bin'),
  PosixPath('../data/html/ped/2022101867.bin'),
  PosixPath('../data/html/ped/2022103576.bin'),
  PosixPath('../data/html/ped/2022102682.bin'),
  PosixPath('../data/html/ped/2022103713.bin'),
  PosixPath('../data/html/ped/2020101920.bin'),
  PosixPath('../data/html/ped/2022104401

In [34]:
#リーディング情報の取得/騎手
for year in [2015,2016,2017,2018,2019,2020,2021,2022, 2023,2024]:
    _ = scraping.scrape_html_leading(
        leading_type="jockey_leading",
        year=year,
        pages=[1, 2, 3, 4],
    )
    
# 2017年のみ5ページ目まである
_ = scraping.scrape_html_leading(
    leading_type="jockey_leading",
    year=2017,
    pages=[1, 2, 3, 4, 5],
)

100%|███████████████████████████████████████████| 4/4 [00:00<00:00, 3225.77it/s]


skipped: ../data/html/leading/jockey_leading/2015_01.bin
skipped: ../data/html/leading/jockey_leading/2015_02.bin
skipped: ../data/html/leading/jockey_leading/2015_03.bin
skipped: ../data/html/leading/jockey_leading/2015_04.bin


100%|███████████████████████████████████████████| 4/4 [00:00<00:00, 4523.38it/s]


skipped: ../data/html/leading/jockey_leading/2016_01.bin
skipped: ../data/html/leading/jockey_leading/2016_02.bin
skipped: ../data/html/leading/jockey_leading/2016_03.bin
skipped: ../data/html/leading/jockey_leading/2016_04.bin


100%|███████████████████████████████████████████| 4/4 [00:00<00:00, 3721.65it/s]


...

skipped: ../data/html/leading/jockey_leading/2023_01.bin
skipped: ../data/html/leading/jockey_leading/2023_02.bin
skipped: ../data/html/leading/jockey_leading/2023_03.bin
skipped: ../data/html/leading/jockey_leading/2023_04.bin


100%|██████████████████████████████████████████| 4/4 [00:00<00:00, 14550.92it/s]


skipped: ../data/html/leading/jockey_leading/2024_01.bin
skipped: ../data/html/leading/jockey_leading/2024_02.bin
skipped: ../data/html/leading/jockey_leading/2024_03.bin
skipped: ../data/html/leading/jockey_leading/2024_04.bin


100%|███████████████████████████████████████████| 5/5 [00:00<00:00, 7505.91it/s]

skipped: ../data/html/leading/jockey_leading/2017_01.bin
skipped: ../data/html/leading/jockey_leading/2017_02.bin
skipped: ../data/html/leading/jockey_leading/2017_03.bin
skipped: ../data/html/leading/jockey_leading/2017_04.bin
skipped: ../data/html/leading/jockey_leading/2017_05.bin


In [35]:
# スクレイピングしたhtmlファイルのパスを取得
html_paths_jockey_leading = list(
    (scraping.HTML_LEADING_DIR / "jockey_leading").glob("*.bin")
)
# rawデータの作成
jockey_leading = create_rawdf.create_jockey_leading(html_paths_jockey_leading)

  0%|          | 0/41 [00:00<?, ?it/s]

In [36]:
#調教師
for year in [2015,2016,2017,2018,2019,2020,2021,2022, 2023,2024]:
    _ = scraping.scrape_html_leading(
        leading_type="trainer_leading",
        year=year,
        pages=[1, 2, 3, 4, 5],
    )
    
# 2018年のみ6ページ目まである
_ = scraping.scrape_html_leading(
    leading_type="trainer_leading",
    year=2018,
    pages=[1, 2, 3, 4, 5, 6],
)

100%|███████████████████████████████████████████| 5/5 [00:00<00:00, 6362.72it/s]


skipped: ../data/html/leading/trainer_leading/2015_01.bin
skipped: ../data/html/leading/trainer_leading/2015_02.bin
skipped: ../data/html/leading/trainer_leading/2015_03.bin
skipped: ../data/html/leading/trainer_leading/2015_04.bin
skipped: ../data/html/leading/trainer_leading/2015_05.bin


100%|███████████████████████████████████████████| 5/5 [00:00<00:00, 9450.89it/s]


skipped: ../data/html/leading/trainer_leading/2016_01.bin
skipped: ../data/html/leading/trainer_leading/2016_02.bin
skipped: ../data/html/leading/trainer_leading/2016_03.bin
skipped: ../data/html/leading/trainer_leading/2016_04.bin
skipped: ../data/html/leading/trainer_leading/2016_05.bin


100%|██████████████████████████████████████████| 5/5 [00:00<00:00, 13477.84it/s]


...

skipped: ../data/html/leading/trainer_leading/2023_01.bin
skipped: ../data/html/leading/trainer_leading/2023_02.bin
skipped: ../data/html/leading/trainer_leading/2023_03.bin
skipped: ../data/html/leading/trainer_leading/2023_04.bin
skipped: ../data/html/leading/trainer_leading/2023_05.bin


100%|███████████████████████████████████████████| 5/5 [00:00<00:00, 6630.26it/s]


skipped: ../data/html/leading/trainer_leading/2024_01.bin
skipped: ../data/html/leading/trainer_leading/2024_02.bin
skipped: ../data/html/leading/trainer_leading/2024_03.bin
skipped: ../data/html/leading/trainer_leading/2024_04.bin
skipped: ../data/html/leading/trainer_leading/2024_05.bin


100%|██████████████████████████████████████████| 6/6 [00:00<00:00, 10490.13it/s]

skipped: ../data/html/leading/trainer_leading/2018_01.bin
skipped: ../data/html/leading/trainer_leading/2018_02.bin
skipped: ../data/html/leading/trainer_leading/2018_03.bin
skipped: ../data/html/leading/trainer_leading/2018_04.bin
skipped: ../data/html/leading/trainer_leading/2018_05.bin
skipped: ../data/html/leading/trainer_leading/2018_06.bin


In [37]:
# スクレイピングしたhtmlファイルのパスを取得
html_paths_trainer_leading = list(
    (scraping.HTML_LEADING_DIR / "trainer_leading").glob("*.bin")
)
# rawデータの作成
trainer_leading = create_rawdf.create_trainer_leading(html_paths_trainer_leading)

  0%|          | 0/51 [00:00<?, ?it/s]

In [38]:
#種牡馬
for year in range(2015, 2025):
    _ = scraping.scrape_html_leading(
        leading_type="sire_leading",
        year=year,
        pages=range(1, 10),
    )

100%|██████████████████████████████████████████| 9/9 [00:00<00:00, 14358.59it/s]


skipped: ../data/html/leading/sire_leading/2015_01.bin
skipped: ../data/html/leading/sire_leading/2015_02.bin
skipped: ../data/html/leading/sire_leading/2015_03.bin
skipped: ../data/html/leading/sire_leading/2015_04.bin
skipped: ../data/html/leading/sire_leading/2015_05.bin
skipped: ../data/html/leading/sire_leading/2015_06.bin
skipped: ../data/html/leading/sire_leading/2015_07.bin
skipped: ../data/html/leading/sire_leading/2015_08.bin
skipped: ../data/html/leading/sire_leading/2015_09.bin


100%|███████████████████████████████████████████| 9/9 [00:00<00:00, 4573.39it/s]


skipped: ../data/html/leading/sire_leading/2016_01.bin
skipped: ../data/html/leading/sire_leading/2016_02.bin
skipped: ../data/html/leading/sire_leading/2016_03.bin
skipped: ../data/html/leading/sire_leading/2016_04.bin
skipped: ../data/html/leading/sire_leading/2016_05.bin
skipped: ../data/html/leading/sire_leading/2016_06.bin
skipped: ../data/html/leading/sire_leading/2016_07.bin
skipped: ../data/html/leading/sire_leading/2016_08.bin
skipped: ../data/html/leading/sire_leading/2016_09.bin


100%|███████████████████████████████████████████| 9/9 [00:00<00:00, 4275.06it/s]


...

skipped: ../data/html/leading/sire_leading/2022_01.bin
skipped: ../data/html/leading/sire_leading/2022_02.bin
skipped: ../data/html/leading/sire_leading/2022_03.bin
skipped: ../data/html/leading/sire_leading/2022_04.bin
skipped: ../data/html/leading/sire_leading/2022_05.bin
skipped: ../data/html/leading/sire_leading/2022_06.bin
skipped: ../data/html/leading/sire_leading/2022_07.bin
skipped: ../data/html/leading/sire_leading/2022_08.bin
skipped: ../data/html/leading/sire_leading/2022_09.bin


100%|██████████████████████████████████████████| 9/9 [00:00<00:00, 21076.90it/s]


skipped: ../data/html/leading/sire_leading/2023_01.bin
skipped: ../data/html/leading/sire_leading/2023_02.bin
skipped: ../data/html/leading/sire_leading/2023_03.bin
skipped: ../data/html/leading/sire_leading/2023_04.bin
skipped: ../data/html/leading/sire_leading/2023_05.bin
skipped: ../data/html/leading/sire_leading/2023_06.bin
skipped: ../data/html/leading/sire_leading/2023_07.bin
skipped: ../data/html/leading/sire_leading/2023_08.bin
skipped: ../data/html/leading/sire_leading/2023_09.bin


100%|██████████████████████████████████████████| 9/9 [00:00<00:00, 18706.01it/s]

skipped: ../data/html/leading/sire_leading/2024_01.bin
skipped: ../data/html/leading/sire_leading/2024_02.bin
skipped: ../data/html/leading/sire_leading/2024_03.bin
skipped: ../data/html/leading/sire_leading/2024_04.bin
skipped: ../data/html/leading/sire_leading/2024_05.bin
skipped: ../data/html/leading/sire_leading/2024_06.bin
skipped: ../data/html/leading/sire_leading/2024_07.bin
skipped: ../data/html/leading/sire_leading/2024_08.bin
skipped: ../data/html/leading/sire_leading/2024_09.bin


In [39]:
# スクレイピングしたhtmlファイルのパスを取得
html_paths_sire_leading = list(
    (scraping.HTML_LEADING_DIR / "sire_leading").glob("*.bin")
)

# rawデータの作成
sire_leading = create_rawdf.create_sire_leading(html_paths_sire_leading)

  0%|          | 0/90 [00:00<?, ?it/s]

In [40]:
for year in range(2015, 2025):
    _ = scraping.scrape_html_leading(
        leading_type="bms_leading",
        year=year,
        pages=range(1, 10),
    )

100%|███████████████████████████████████████████| 9/9 [00:00<00:00, 9933.88it/s]


skipped: ../data/html/leading/bms_leading/2015_01.bin
skipped: ../data/html/leading/bms_leading/2015_02.bin
skipped: ../data/html/leading/bms_leading/2015_03.bin
skipped: ../data/html/leading/bms_leading/2015_04.bin
skipped: ../data/html/leading/bms_leading/2015_05.bin
skipped: ../data/html/leading/bms_leading/2015_06.bin
skipped: ../data/html/leading/bms_leading/2015_07.bin
skipped: ../data/html/leading/bms_leading/2015_08.bin
skipped: ../data/html/leading/bms_leading/2015_09.bin


100%|██████████████████████████████████████████| 9/9 [00:00<00:00, 13960.33it/s]


skipped: ../data/html/leading/bms_leading/2016_01.bin
skipped: ../data/html/leading/bms_leading/2016_02.bin
skipped: ../data/html/leading/bms_leading/2016_03.bin
skipped: ../data/html/leading/bms_leading/2016_04.bin
skipped: ../data/html/leading/bms_leading/2016_05.bin
skipped: ../data/html/leading/bms_leading/2016_06.bin
skipped: ../data/html/leading/bms_leading/2016_07.bin
skipped: ../data/html/leading/bms_leading/2016_08.bin
skipped: ../data/html/leading/bms_leading/2016_09.bin


100%|██████████████████████████████████████████| 9/9 [00:00<00:00, 10503.27it/s]


...

skipped: ../data/html/leading/bms_leading/2022_01.bin
skipped: ../data/html/leading/bms_leading/2022_02.bin
skipped: ../data/html/leading/bms_leading/2022_03.bin
skipped: ../data/html/leading/bms_leading/2022_04.bin
skipped: ../data/html/leading/bms_leading/2022_05.bin
skipped: ../data/html/leading/bms_leading/2022_06.bin
skipped: ../data/html/leading/bms_leading/2022_07.bin
skipped: ../data/html/leading/bms_leading/2022_08.bin
skipped: ../data/html/leading/bms_leading/2022_09.bin


100%|███████████████████████████████████████████| 9/9 [00:00<00:00, 7037.42it/s]


skipped: ../data/html/leading/bms_leading/2023_01.bin
skipped: ../data/html/leading/bms_leading/2023_02.bin
skipped: ../data/html/leading/bms_leading/2023_03.bin
skipped: ../data/html/leading/bms_leading/2023_04.bin
skipped: ../data/html/leading/bms_leading/2023_05.bin
skipped: ../data/html/leading/bms_leading/2023_06.bin
skipped: ../data/html/leading/bms_leading/2023_07.bin
skipped: ../data/html/leading/bms_leading/2023_08.bin
skipped: ../data/html/leading/bms_leading/2023_09.bin


100%|███████████████████████████████████████████| 9/9 [00:00<00:00, 8197.34it/s]

skipped: ../data/html/leading/bms_leading/2024_01.bin
skipped: ../data/html/leading/bms_leading/2024_02.bin
skipped: ../data/html/leading/bms_leading/2024_03.bin
skipped: ../data/html/leading/bms_leading/2024_04.bin
skipped: ../data/html/leading/bms_leading/2024_05.bin
skipped: ../data/html/leading/bms_leading/2024_06.bin
skipped: ../data/html/leading/bms_leading/2024_07.bin
skipped: ../data/html/leading/bms_leading/2024_08.bin
skipped: ../data/html/leading/bms_leading/2024_09.bin


In [41]:
# スクレイピングしたhtmlファイルのパスを取得
html_paths_bms_leading = list(
    (scraping.HTML_LEADING_DIR / "bms_leading").glob("*.bin")
)

# rawデータの作成
bms_leading = create_rawdf.create_bms_leading(html_paths_bms_leading)

  0%|          | 0/90 [00:00<?, ?it/s]

### 実際に予測する際の事前準備

- 実際の予測時の処理
- 予測したいレース前日などに出走馬が発表されたら、以下の事前準備を行う

In [5]:
# 予測母集団の作成
#予測したい日を入力してください
prediction_population = create_prediction_population.create(kaisai_date="20250113")

scraping race_id_list...


100%|█████████████████████████████████████████████| 1/1 [00:22<00:00, 22.12s/it]

scraping horse_id_list...


  0%|          | 0/24 [00:00<?, ?it/s]

In [6]:
prediction_population 

,date,race_id,horse_id
0,2025-01-13,202506010501,2022103450
1,2025-01-13,202506010501,2022104017
2,2025-01-13,202506010501,2022107197
3,2025-01-13,202506010501,2022103461
4,2025-01-13,202506010501,2022102673
...,...,...,...
9,2025-01-13,202507010512,2019104536
10,2025-01-13,202507010512,2020101031
11,2025-01-13,202507010512,2019100578
12,2025-01-13,202507010512,2020104825


In [7]:
# 当日出走馬のhorse_idリスト
horse_id_list = prediction_population["horse_id"].unique()

In [8]:
## 馬の過去成績取得 更新している可能性が高いため
html_paths_horse = scraping.scrape_html_horse(horse_id_list=horse_id_list, skip=False)

100%|█████████████████████████████████████████| 369/369 [07:53<00:00,  1.28s/it]


Errors occurred for the following horse_ids:
[]


In [61]:
html_paths_horse 

[PosixPath('../data/html/horse/2022103758.bin'),
 PosixPath('../data/html/horse/2022106861.bin'),
 PosixPath('../data/html/horse/2022103333.bin'),
 PosixPath('../data/html/horse/2022106669.bin'),
 PosixPath('../data/html/horse/2022106454.bin'),
 PosixPath('../data/html/horse/2022106979.bin'),
 PosixPath('../data/html/horse/2022104211.bin'),
 PosixPath('../data/html/horse/2022101185.bin'),
 PosixPath('../data/html/horse/2022100039.bin'),
 PosixPath('../data/html/horse/2022104334.bin'),
 PosixPath('../data/html/horse/2022101587.bin'),
 PosixPath('../data/html/horse/2022106410.bin'),
 PosixPath('../data/html/horse/2022101258.bin'),
 PosixPath('../data/html/horse/2022102321.bin'),
 PosixPath('../data/html/horse/2022100050.bin'),
 PosixPath('../data/html/horse/2022101908.bin'),
 PosixPath('../data/html/horse/2022100587.bin'),
 PosixPath('../data/html/horse/2022101685.bin'),
 PosixPath('../data/html/horse/2022106431.bin'),
 PosixPath('../data/html/horse/2022100738.bin'),
 PosixPath('../data/

In [ ]:
#エラー修正

In [70]:
# バイナリファイルの読み込み例
with open(HTML_HORSE_DIR /'2022101314.bin', 'rb') as file:
    data = file.read()  

In [71]:
df = pd.read_html(data)[2]
df
#取れてない方

,日付,開催,天 気,R,レース名,映 像,頭 数,枠 番,馬 番,オ ッ ズ,...,着差,ﾀｲﾑ 指数,通過,ペース,上り,馬体重,厩舎 ｺﾒﾝﾄ,備考,勝ち馬 (2着馬),賞金
0,2024/11/30,4中京1,晴,8,3歳以上1勝クラス,NaN,16,3,6,11.2,...,2.8,**,4-8-9,34.6-35.8,37.5,586(+14),NaN,NaN,ダズリングブレイヴ,NaN
1,2024/06/29,3小倉1,曇,8,3歳以上1勝クラス,NaN,12,1,1,17.1,...,1.0,**,1-1-1-2,35.1-37.0,38.0,572(+6),NaN,NaN,カズミクラーシュ,NaN
2,2024/03/16,1阪神7,晴,11,若葉S(L),NaN,9,8,9,36.3,...,3.5,**,3-3-2-2,36.1-34.6,37.9,566(0),NaN,NaN,ミスタージーティー,NaN
3,2024/01/28,2京都2,晴,5,3歳新馬,NaN,13,4,4,4.8,...,0.0,**,2-2-2-2,37.6-35.6,35.6,566(0),NaN,NaN,(ジャスティンダイス),620.0


In [4]:
# バイナリファイルの読み込み例
with open(HTML_HORSE_DIR /'2022100896.bin', 'rb') as file:
    data2 = file.read()  

In [21]:
df2 = pd.read_html(data2)[2]
df2

#取れてる方

,日付,開催,天 気,R,レース名,映 像,頭 数,枠 番,馬 番,オ ッ ズ,...,着差,ﾀｲﾑ 指数,通過,ペース,上り,馬体重,厩舎 ｺﾒﾝﾄ,備考,勝ち馬 (2着馬),賞金
0,2024/10/19,4東京5,晴,9,アイビーS(L),NaN,8,1,1,41.2,...,1.9,**,6-6-6,35.7-34.3,34.5,404(-6),NaN,NaN,マスカレードボール,NaN
1,2024/09/22,4中山7,曇,5,2歳新馬,NaN,14,8,13,23.5,...,-0.3,**,1-1-1-1,37.3-34.8,34.8,410(0),NaN,NaN,(アルティテュード),720.0


In [87]:
df.columns[0]

np.int64(0)

In [80]:
from pathlib import Path  # PosixPath を使うために pathlib をインポート

# 当日出走馬の過去成績テーブル作成
horse_results_error = create_rawdf.create_horse_results(
    html_path_list=[Path(HTML_HORSE_DIR) /'2021102904.bin'],  # リストとして渡す
    save_filename="horse_results_error.csv",
)


  0%|          | 0/1 [00:00<?, ?it/s]

In [81]:
horse_results_error

,horse_id,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,人気,着順,騎手,斤量,距離,馬場,馬場指数,タイム,着差,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金
0,2021102904,2024/11/30,4中京1,晴,8,3歳以上1勝クラス,NaN,16,3,6,11.2,5,14,鮫島克駿,57,芝1600,良,**,1:36.3,2.8,**,4-8-9,34.6-35.8,37.5,586(+14),NaN,NaN,ダズリングブレイヴ,NaN
1,2021102904,2024/06/29,3小倉1,曇,8,3歳以上1勝クラス,NaN,12,1,1,17.1,4,7,西村淳也,55,芝1800,良,**,1:48.9,1.0,**,1-1-1-2,35.1-37.0,38.0,572(+6),NaN,NaN,カズミクラーシュ,NaN
2,2021102904,2024/03/16,1阪神7,晴,11,若葉S(L),NaN,9,8,9,36.3,8,9,池添謙一,57,芝2000,良,**,2:03.2,3.5,**,3-3-2-2,36.1-34.6,37.9,566(0),NaN,NaN,ミスタージーティー,NaN
3,2021102904,2024/01/28,2京都2,晴,5,3歳新馬,NaN,13,4,4,4.8,2,1,池添謙一,57,芝2000,良,**,2:03.6,0.0,**,2-2-2-2,37.6-35.6,35.6,566(0),NaN,NaN,(ジャスティンダイス),620.0


In [9]:
# 当日出走馬の過去成績テーブル作成
horse_results_prediction = create_rawdf.create_horse_results(
    html_path_list=html_paths_horse,
    save_filename="horse_results_prediction.csv",
)

  0%|          | 0/369 [00:00<?, ?it/s]

2022107359 の HTML に3つ目のテーブルがありません
2022102555 の HTML に3つ目のテーブルがありません
2022106889 の HTML に3つ目のテーブルがありません
2022102322 の HTML に3つ目のテーブルがありません
2022102767 の HTML に3つ目のテーブルがありません
2022102512 の HTML に3つ目のテーブルがありません
2022100742 の HTML に3つ目のテーブルがありません
2022102497 の HTML に3つ目のテーブルがありません
2022104306 の HTML に3つ目のテーブルがありません
2022107277 の HTML に3つ目のテーブルがありません
2022105090 の HTML に3つ目のテーブルがありません
2022104468 の HTML に3つ目のテーブルがありません
2022106268 の HTML に3つ目のテーブルがありません
2022101218 の HTML に3つ目のテーブルがありません
2022102681 の HTML に3つ目のテーブルがありません
2022102272 の HTML に3つ目のテーブルがありません
2022103122 の HTML に3つ目のテーブルがありません
2022103078 の HTML に3つ目のテーブルがありません
2022100595 の HTML に3つ目のテーブルがありません
2022102202 の HTML に3つ目のテーブルがありません
2022107411 の HTML に3つ目のテーブルがありません
2022102187 の HTML に3つ目のテーブルがありません
2022102360 の HTML に3つ目のテーブルがありません
2022105688 の HTML に3つ目のテーブルがありません
2022103198 の HTML に3つ目のテーブルがありません
2022103887 の HTML に3つ目のテーブルがありません
2022103631 の HTML に3つ目のテーブルがありません
2022103056 の HTML に3つ目のテーブルがありません
2022100534 の HTML に3つ目のテーブルがありません
2022100102 の H

In [10]:
#血統データ取得
# 当日出走馬の血統をスクレイピング
html_paths_peds = scraping.scrape_html_ped(
    horse_id_list=horse_id_list,
    save_dir=Path(HTML_PED_DIR)
)

  1%|▌                                          | 5/369 [00:00<00:08, 45.10it/s]

skipped: 2022103450
skipped: 2022104017
skipped: 2022107197
skipped: 2022103461
skipped: 2022104072
skipped: 2022100209
skipped: 2022100547
skipped: 2022105749


  5%|██                                        | 18/369 [00:00<00:08, 43.03it/s]

skipped: 2022102592
skipped: 2022101604
skipped: 2022102267
skipped: 2022102975
skipped: 2022100627
skipped: 2022102834
skipped: 2022107021
skipped: 2022102558
skipped: 2022105699
skipped: 2022110126
skipped: 2022102260
skipped: 2022101801
skipped: 2022103514
skipped: 2022102606
skipped: 2022105648
skipped: 2022103994
skipped: 2022100949
skipped: 2022107138
skipped: 2022102063
skipped: 2022105377
skipped: 2022103401


  9%|███▊                                      | 33/369 [00:00<00:06, 52.96it/s]

...

skipped: 2022100838
skipped: 2022110065


 61%|█████████████████████████                | 226/369 [00:09<00:06, 20.50it/s]

skipped: 2022103378
skipped: 2022106565


100%|█████████████████████████████████████████| 369/369 [00:16<00:00, 22.52it/s]

skipped: 2021105604
skipped: 2021100604
skipped: 2021102433
skipped: 2020100589
skipped: 2021100652
skipped: 2021102469
skipped: 2021103837
skipped: 2020104791
skipped: 2020103274
skipped: 2021103583
skipped: 2021100794
skipped: 2021105613
skipped: 2021100680
skipped: 2021104512
skipped: 2021102399
skipped: 2021103932
skipped: 2021105865
skipped: 2021105348
skipped: 2021106128
skipped: 2021105765
skipped: 2020102736
skipped: 2021100651
skipped: 2021103002
skipped: 2019101618
skipped: 2020104759
skipped: 2020102570
skipped: 2021101155
skipped: 2020102574
skipped: 2020100230
skipped: 2021106194
skipped: 2020103048
skipped: 2020104000
skipped: 2020104491
skipped: 2021105875
skipped: 2021100376
skipped: 2021106707
skipped: 2020101563
skipped: 2020102959
skipped: 2021103574
skipped: 2020101268
skipped: 2021105142
skipped: 2021105637
skipped: 2020102664
skipped: 2016106594
skipped: 2021101201
skipped: 2021106628
skipped: 2021104952
skipped: 2021105831
skipped: 2020100978
skipped: 2021103586


In [11]:
# skipされたものも含めて全てのhtmlファイルのパスを取得
html_paths_peds = [scraping.HTML_PED_DIR / f"{horse_id}.bin" for horse_id in horse_id_list]

In [12]:
# 当日出走馬の血統テーブル作成
peds_prediction = create_rawdf.create_peds(
    html_path_list=html_paths_peds,
    save_filename="peds_prediction.csv",
)

  0%|          | 0/369 [00:00<?, ?it/s]

### 馬場状態のための過去データ収集

In [207]:
TMP_DIR2 = scraping.DATA_DIR / "tmp_predict"

In [208]:
# 開催日一覧の取得、行うレースの月を含む1ヶ月前をとっておく、実際に使用するのは12日前のレースのみ
kaisai_date_list_prediction = scraping.scrape_kaisai_date(
    from_="2024-12", to_="2025-01", save_dir=TMP_DIR2
)

100%|█████████████████████████████████████████████| 2/2 [00:02<00:00,  1.28s/it]


In [209]:
kaisai_date_list_prediction

['20241201',
 '20241207',
 '20241208',
 '20241214',
 '20241215',
 '20241221',
 '20241222',
 '20241228',
 '20250105',
 '20250106',
 '20250111',
 '20250112',
 '20250113']

In [210]:
cut_index = kaisai_date_list_prediction.index("20250111")

# リストをスライスして削除
kaisai_date_list_prediction = kaisai_date_list_prediction[:cut_index]

In [211]:
kaisai_date_list_prediction

['20241201',
 '20241207',
 '20241208',
 '20241214',
 '20241215',
 '20241221',
 '20241222',
 '20241228',
 '20250105',
 '20250106']

In [212]:
# スクレイピング対象レースのid取得
race_id_list_prediction = scraping.scrape_race_id_list(
    kaisai_date_list_prediction, save_dir=TMP_DIR2
)

  0%|                                                    | 0/10 [00:00<?, ?it/s]

skipped: 202406050201
skipped: 202406050202
skipped: 202406050203
skipped: 202406050204
skipped: 202406050205
skipped: 202406050206
skipped: 202406050207
skipped: 202406050208
skipped: 202406050209
skipped: 202406050210
skipped: 202406050211
skipped: 202406050212
skipped: 202408070201
skipped: 202408070202
skipped: 202408070203
skipped: 202408070204
skipped: 202408070205
skipped: 202408070206
skipped: 202408070207


 10%|████▍                                       | 1/10 [01:23<12:34, 83.85s/it]

skipped: 202408070208
skipped: 202408070209
skipped: 202408070210
skipped: 202408070211
skipped: 202408070212
skipped: 202407040201
skipped: 202407040202
skipped: 202407040203
skipped: 202407040204
skipped: 202407040205
skipped: 202407040206
skipped: 202407040207
skipped: 202407040208
skipped: 202407040209
skipped: 202407040210
skipped: 202407040211
skipped: 202407040212
skipped: 202406050301
skipped: 202406050302
skipped: 202406050303
skipped: 202406050304
skipped: 202406050305
skipped: 202406050306
skipped: 202406050307
skipped: 202406050308
skipped: 202406050309
skipped: 202406050310
skipped: 202406050311
skipped: 202406050312
skipped: 202408070301
skipped: 202408070302
skipped: 202408070303
skipped: 202408070304
skipped: 202408070305
skipped: 202408070306
skipped: 202408070307
skipped: 202408070308


 20%|████████▊                                   | 2/10 [01:27<04:55, 36.92s/it]

...

 80%|███████████████████████████████████▏        | 8/10 [01:47<00:11,  5.96s/it]

skipped: 202408070911
skipped: 202408070912
skipped: 202506010101
skipped: 202506010102
skipped: 202506010103
skipped: 202506010104
skipped: 202506010105
skipped: 202506010106
skipped: 202506010107
skipped: 202506010108
skipped: 202506010109
skipped: 202506010110


 90%|███████████████████████████████████████▌    | 9/10 [01:51<00:05,  5.36s/it]

skipped: 202506010111
skipped: 202506010112
skipped: 202507010101
skipped: 202507010102
skipped: 202507010103
skipped: 202507010104
skipped: 202507010105
skipped: 202507010106
skipped: 202507010107
skipped: 202507010108
skipped: 202507010109
skipped: 202507010110
skipped: 202507010111
skipped: 202507010112


100%|███████████████████████████████████████████| 10/10 [01:54<00:00, 11.48s/it]


In [213]:
race_id_list_prediction

['202405040101',
 '202405040102',
 '202405040103',
 '202405040104',
 '202405040105',
 '202405040106',
 '202405040107',
 '202405040108',
 '202405040109',
 '202405040110',
 '202405040111',
 '202405040112',
 '202408050101',
 '202408050102',
 '202408050103',
 '202408050104',
 '202408050105',
 '202408050106',
 '202408050107',
 '202408050108',
 '202408050109',
 '202408050110',
 '202408050111',
 '202408050112',
 '202404040101',
 '202404040102',
 '202404040103',
 '202404040104',
 '202404040105',
 '202404040106',
 '202404040107',
 '202404040108',
 '202404040109',
 '202404040110',
 '202404040111',
 '202404040112',
 '202405040201',
 '202405040202',
 '202405040203',
 '202405040204',
 '202405040205',
 '202405040206',
 '202405040207',
 '202405040208',
 '202405040209',
 '202405040210',
 '202405040211',
 '202405040212',
 '202408050201',
 '202408050202',
 '202408050203',
 '202408050204',
 '202408050205',
 '202408050206',
 '202408050207',
 '202408050208',
 '202408050209',
 '202408050210',
 '202408050211

In [214]:
# raceページのhtmlをスクレイピング
html_paths_race_prediction = scraping_prediction.scrape_html_race(race_id_list=race_id_list_prediction)

  0%|          | 0/887 [00:00<?, ?it/s]

skipped: 202405040101
skipped: 202405040102
skipped: 202405040103
skipped: 202405040104
skipped: 202405040105
skipped: 202405040106
skipped: 202405040107
skipped: 202405040108
skipped: 202405040109
skipped: 202405040110
skipped: 202405040111
skipped: 202405040112
skipped: 202408050101
skipped: 202408050102
skipped: 202408050103
skipped: 202408050104
skipped: 202408050105
skipped: 202408050106
skipped: 202408050107
skipped: 202408050108
skipped: 202408050109
skipped: 202408050110
skipped: 202408050111
skipped: 202408050112
skipped: 202404040101
skipped: 202404040102
skipped: 202404040103
skipped: 202404040104
skipped: 202404040105
skipped: 202404040106
skipped: 202404040107
skipped: 202404040108
skipped: 202404040109
skipped: 202404040110
skipped: 202404040111
skipped: 202404040112
skipped: 202405040201
skipped: 202405040202
skipped: 202405040203
skipped: 202405040204
skipped: 202405040205
skipped: 202405040206
skipped: 202405040207
skipped: 202405040208
skipped: 202405040209
skipped: 2

In [215]:
html_paths_race_prediction

[PosixPath('../data/html/race2/202506010201.bin'),
 PosixPath('../data/html/race2/202506010202.bin'),
 PosixPath('../data/html/race2/202506010203.bin'),
 PosixPath('../data/html/race2/202506010204.bin'),
 PosixPath('../data/html/race2/202506010205.bin'),
 PosixPath('../data/html/race2/202506010206.bin'),
 PosixPath('../data/html/race2/202506010207.bin'),
 PosixPath('../data/html/race2/202506010208.bin'),
 PosixPath('../data/html/race2/202506010209.bin'),
 PosixPath('../data/html/race2/202506010210.bin'),
 PosixPath('../data/html/race2/202506010211.bin'),
 PosixPath('../data/html/race2/202506010212.bin'),
 PosixPath('../data/html/race2/202507010201.bin'),
 PosixPath('../data/html/race2/202507010202.bin'),
 PosixPath('../data/html/race2/202507010203.bin'),
 PosixPath('../data/html/race2/202507010204.bin'),
 PosixPath('../data/html/race2/202507010205.bin'),
 PosixPath('../data/html/race2/202507010206.bin'),
 PosixPath('../data/html/race2/202507010207.bin'),
 PosixPath('../data/html/race2/

In [216]:
# 途中で処理が途切れるなどした場合は、直接htmlのファイルパスを取得
html_paths_race_prediction = [
    scraping_prediction.HTML_RACE_DIR / f"{race_id}.bin" for race_id in race_id_list_prediction
]

In [217]:
results_pre = scraping_prediction.create_results(html_path_list=html_paths_race_prediction)

  0%|          | 0/887 [00:00<?, ?it/s]

table not found at 202405040108
table not found at 202404040101
table not found at 202405040203
table not found at 202404040204
table not found at 202408050304
table not found at 202408050309
table not found at 202404040304
table not found at 202408050402
table not found at 202408050404
table not found at 202408050412
table not found at 202404040406
table not found at 202408050601
table not found at 202408050605
table not found at 202404040602
table not found at 202408050707
table not found at 202408050712
table not found at 202404040701
table not found at 202408050812
table not found at 202403030104
table not found at 202408060202
table not found at 202403030202
table not found at 202403030203
table not found at 202403030208
table not found at 202405050412
table not found at 202408060402
table not found at 202403030411
table not found at 202408060506
table not found at 202408060604
table not found at 202408060606
table not found at 202403030603
table not found at 202403030606
table no

In [48]:
import re

# メッセージリスト
messages = [
    "table not found at 202405040108",
    "table not found at 202404040101",
    "table not found at 202405040203",
    "table not found at 202404040204",
    "table not found at 202408050304",
    "table not found at 202408050309",
    "table not found at 202404040304",
    "table not found at 202408050402",
    "table not found at 202408050404",
    "table not found at 202408050412",
    "table not found at 202404040406",
    "table not found at 202408050601",
    "table not found at 202408050605",
    "table not found at 202404040602",
    "table not found at 202408050707",
    "table not found at 202408050712",
    "table not found at 202404040701",
    "table not found at 202408050812",
    "table not found at 202403030104",
    "table not found at 202408060202",
    "table not found at 202403030202",
    "table not found at 202403030203",
    "table not found at 202403030208",
    "table not found at 202405050412",
    "table not found at 202408060402",
    "table not found at 202403030411",
    "table not found at 202408060506",
    "table not found at 202408060604",
    "table not found at 202408060606",
    "table not found at 202403030603",
    "table not found at 202403030606",
    "table not found at 202408070305",
    "table not found at 202408070504",
    "table not found at 202406050604",
    "table not found at 202408070602",
    "table not found at 202406050710",
    "table not found at 202408070805",
    "table not found at 202408070904"

]

# 正規表現で数字部分を抽出
missing_tables = [re.search(r'\d{12}', message).group() for message in messages]

# 抽出したリストを表示
print(missing_tables)


['202405040108', '202404040101', '202405040203', '202404040204', '202408050304', '202408050309', '202404040304', '202408050402', '202408050404', '202408050412', '202404040406', '202408050601', '202408050605', '202404040602', '202408050707', '202408050712', '202404040701', '202408050812', '202403030104', '202408060202', '202403030202', '202403030203', '202403030208', '202405050412', '202408060402', '202403030411', '202408060506', '202408060604', '202408060606', '202403030603', '202403030606', '202408070305', '202408070504', '202406050604', '202408070602', '202406050710', '202408070805', '202408070904']


In [228]:
#エラー引き起こしたレースIDを追加
import re
from pathlib import Path
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

DATA_DIR = Path("..", "data")
RAWDF_DIR = DATA_DIR / "rawdf2"

# 例: 取得時にエラーが発生したrace_idのリスト
error_race_ids =missing_tables
error_race_ids =["202506010208"]
# 実際のエラーのrace_idをリストに格納

dfs = {}

# エラーのrace_idリストをループ処理
for error_race_id in tqdm(error_race_ids):
    try:
        html_path = DATA_DIR / "html" / "race2" / f"{error_race_id}.bin"  # 対応するHTMLファイルのパス
        with open(html_path, "rb") as f:
            
            race_id = html_path.stem
            html = (
                f.read()
                .replace(b"<diary_snap_cut>", b"")
                .replace(b"</diary_snap_cut>", b"")
            )
            soup = BeautifulSoup(html, "lxml").find(
                "table", class_="race_table_01 nk_tb_common"
            )
            df = pd.read_html(html)[0]

        # horse_id列追加
        horse_id_list = []
        a_list = soup.find_all("a", href=re.compile(r"^/horse/"))
        for a in a_list:
            horse_id = re.findall(r"\d{10}", a["href"])[0]
            horse_id_list.append(horse_id)
        df["horse_id"] = horse_id_list

        # jockey_id列追加
        jockey_id_list = []
        a_list = soup.find_all("a", href=re.compile(r"^/jockey/"))
        for a in a_list:
            jockey_id = re.findall(r"\d{5}", a["href"])[0]
            jockey_id_list.append(jockey_id)
        df["jockey_id"] = jockey_id_list

        # trainer_id列追加
        trainer_id_list = []
        a_list = soup.find_all("a", href=re.compile(r"^/trainer/"))
        for a in a_list:
            trainer_id = re.findall(r"\d{5}", a["href"])[0]
            trainer_id_list.append(trainer_id)
        df["trainer_id"] = trainer_id_list

        # 最初の列にrace_idを挿入
        df.insert(0, "race_id", error_race_id)
        dfs[error_race_id] = df
        # owner_id列追加
        owner_id_list = []
        a_list = soup.find_all("a", href=re.compile(r"^/owner/"))
        for a in a_list:
            owner_id = re.findall(r"\d{6}", a["href"])[0]
            owner_id_list.append(owner_id)
        df["owner_id"] = owner_id_list

    except IndexError:
        print(f"table not found at {error_race_id}")
        continue


# 全てのデータフレームを結合
concat_df = pd.concat(dfs.values())
concat_df.columns = concat_df.columns.str.replace(" ", "")

# 結果を保存
update_rawdf(
    concat_df,
    key="race_id",
    save_filename="results.csv",
    save_dir=RAWDF_DIR,
)
def update_rawdf(
    new_df: pd.DataFrame,
    key: str,
    save_filename: str,
    save_dir: Path = RAWDF_DIR,
) -> None:
    """
    既存のrawdfに新しいデータを追加して保存する関数。
    """
    save_dir.mkdir(parents=True, exist_ok=True)
    if (save_dir / save_filename).exists():
        old_df = pd.read_csv(save_dir / save_filename, sep="\t", dtype={f"{key}": str})
        # 念の為、key列をstr型に変換
        new_df[key] = new_df[key].astype(str)
        df = pd.concat([old_df[~old_df[key].isin(new_df[key])], new_df])
        df.to_csv(save_dir / save_filename, sep="\t", index=False)
    else:
        # ファイルが存在しない場合は単にそのまま保存
        new_df.to_csv(save_dir / save_filename, sep="\t", index=False)
# error_race_id = error_race_ids[0]
# html_path = DATA_DIR /"html"/ "race2"/ f"{error_race_id}.bin"  # 対応するHTMLファイルのパス
# with open(html_path, "rb") as f:
#     html = (
#         f.read()
#         .replace(b"<diary_snap_cut>", b"")
#         .replace(b"</diary_snap_cut>", b"")
#     )
#     soup = BeautifulSoup(html, "lxml").find(
#         "table", class_="race_table_01 nk_tb_common"
#     )
#     df = pd.read_html(html)[0]


# # エラーのrace_idリストをループ処理
# # for error_race_id in tqdm(error_race_ids):
# #     try:
# #         html_path = DATA_DIR /"html"/ "race"/ f"{error_race_ids}.bin"   # 対応するHTMLファイルのパス
# #         with open(html_path, "rb") as f:
# #             html = (
# #                 f.read()
# #                 .replace(b"<diary_snap_cut>", b"")
# #                 .replace(b"</diary_snap_cut>", b"")
# #             )
# #             soup = BeautifulSoup(html, "lxml").find(
# #                 "table", class_="race_table_01 nk_tb_common"
# #             )
# #             df = pd.read_html(html)[0]

#             # horse_id列追加
# horse_id_list = []
# a_list = soup.find_all("a", href=re.compile(r"^/horse/"))
# for a in a_list:
#     horse_id = re.findall(r"\d{10}", a["href"])[0]
#     horse_id_list.append(horse_id)
# df["horse_id"] = horse_id_list

# # jockey_id列追加
# jockey_id_list = []
# a_list = soup.find_all("a", href=re.compile(r"^/jockey/"))
# for a in a_list:
#     jockey_id = re.findall(r"\d{5}", a["href"])[0]
#     jockey_id_list.append(jockey_id)
# df["jockey_id"] = jockey_id_list

# # trainer_id列追加
# trainer_id_list = []
# a_list = soup.find_all("a", href=re.compile(r"^/trainer/"))
# for a in a_list:
#     trainer_id = re.findall(r"\d{5}", a["href"])[0]
#     trainer_id_list.append(trainer_id)
# df["trainer_id"] = trainer_id_list

# # # owner_id列追加
# # owner_id_list = []
# # a_list = soup.find_all("a", href=re.compile(r"^/owner/"))
# # for a in a_list:
# #     owner_id = re.findall(r"\d{6}", a["href"])[0]
# #     owner_id_list.append(owner_id)
# # df["owner_id"] = owner_id_list
# import numpy as np

# # a_list = soup.find_all("a", href=re.compile(r"^/owner/"))
# # for a in a_list:
# #     match = re.findall(r"\d{6}", a["href"])
# #     if match:
# #         owner_id = match[0]
# #     else:
# #         owner_id = np.nan  # 空の場合は NaN を挿入
# #     owner_id_list.append(owner_id)

# # # owner_id_list の長さを df の行数に合わせる
# # if len(owner_id_list) < len(df):
# #     # 足りない分を NaN で埋める
# #     owner_id_list.extend([np.nan] * (len(df) - len(owner_id_list)))
# # elif len(owner_id_list) > len(df):
# #     # 多い場合は df の行数に合わせて切り詰める
# #     owner_id_list = owner_id_list[:len(df)]

# # df["owner_id"] = owner_id_list


# # 最初の列にrace_idを挿入
# df.insert(0, "race_id", error_race_id)
# dfs[error_race_id] = df


# # except IndexError as e:
# #     print(f"table not found at {error_race_id}")
# #     continue
# # except Exception as e:
# #     print(f"An error occurred for race_id {error_race_id}: {e}")
# #     continue

# def update_rawdf(
#     new_df: pd.DataFrame,
#     key: str,
#     save_filename: str,
#     save_dir: Path = RAWDF_DIR,
# ) -> None:
#     """
#     既存のrawdfに新しいデータを追加して保存する関数。
#     """
#     save_dir.mkdir(parents=True, exist_ok=True)
#     if (save_dir / save_filename).exists():
#         old_df = pd.read_csv(save_dir / save_filename, sep="\t", dtype={f"{key}": str})
#         # 念の為、key列をstr型に変換
#         new_df[key] = new_df[key].astype(str)
#         df = pd.concat([old_df[~old_df[key].isin(new_df[key])], new_df])
#         df.to_csv(save_dir / save_filename, sep="\t", index=False)
#     else:
#         # ファイルが存在しない場合は単にそのまま保存
#         new_df.to_csv(save_dir / save_filename, sep="\t", index=False)


# concat_df = pd.concat(dfs.values())
# concat_df.columns = concat_df.columns.str.replace(" ", "")
# update_rawdf(
#     concat_df,
#     key="race_id",
#     save_filename="results.csv",
#     save_dir=RAWDF_DIR,
# )
df

  0%|          | 0/1 [00:00<?, ?it/s]

table not found at 202506010208


ValueError: No objects to concatenate

In [ ]:
#これでとれないのはほぼ障害レースか、前日のレースのみ

In [51]:
html_paths_race_prediction = [
    scraping_prediction.HTML_RACE_DIR / f"{race_id}.bin" for race_id in race_id_list_prediction
]
race_info_pre = scraping_prediction.create_race_info(html_path_list=html_paths_race_prediction)

  0%|          | 0/863 [00:00<?, ?it/s]

'NoneType' object has no attribute 'find' at 202506010101
'NoneType' object has no attribute 'find' at 202506010102
'NoneType' object has no attribute 'find' at 202506010103
'NoneType' object has no attribute 'find' at 202506010104
'NoneType' object has no attribute 'find' at 202506010105
'NoneType' object has no attribute 'find' at 202506010106
'NoneType' object has no attribute 'find' at 202506010107
'NoneType' object has no attribute 'find' at 202506010108
'NoneType' object has no attribute 'find' at 202506010109
'NoneType' object has no attribute 'find' at 202506010110
'NoneType' object has no attribute 'find' at 202506010111
'NoneType' object has no attribute 'find' at 202506010112
'NoneType' object has no attribute 'find' at 202507010101
'NoneType' object has no attribute 'find' at 202507010102
'NoneType' object has no attribute 'find' at 202507010103
'NoneType' object has no attribute 'find' at 202507010104
'NoneType' object has no attribute 'find' at 202507010105
'NoneType' obj